# nanochat pre-training

GPT-2レベルの大規模言語モデルを単一のGPUで事前学習する

## 参考

- [nanochat][0]
- [The FineWeb Datasets: Decanting the Web for the Finest Text Data at Scale][4]
- [Muon: An optimizer for hidden layers in neural networks][1]
- [Muon is Scaling for LLM Training][6]
- [A Spectral Condition for Feature Learning][2]
- [PaLM: Scaling Language Modeling with Pathways][3]
- [DataComp-LM: In search of the next generation of training sets for language models][5]
- [Training Compute-Optimal Large Language Models][7]

[0]: https://github.com/karpathy/nanochat
[1]: https://kellerjordan.github.io/posts/muon/
[2]: https://arxiv.org/pdf/2310.17813
[3]: https://arxiv.org/abs/2204.02311
[4]: https://arxiv.org/abs/2406.17557
[5]: https://arxiv.org/abs/2406.11794
[6]: https://arxiv.org/abs/2502.16982
[7]: https://arxiv.org/abs/2203.15556

## 環境構築

In [ ]:
import torch

assert torch.cuda.is_available(), "GPUが利用できません"
device_name = torch.cuda.get_device_name(0)

if "A100" in device_name:
    gpu_name = "A100" # 40GB -> 67時間~
elif "T4" in device_name:
    gpu_name = "T4" # 16GB -> 忍耐が必要（コンパイルに90分ぐらいかかる）
elif "5090" in device_name:
    gpu_name = "RTX5090" # 32GB -> 74時間~
else:
    raise NotImplementedError # 未検証

# 公式のスクリプト（speedrun.sh）の場合、8台のH100 GPU（VRAM80GB）で4時間かかる（~$100）
# GPT-3級のスクリプト（run1000.sh）の場合、8台のH100 GPUで32時間かかる（~$1000）
gpu_name

In [ ]:
# お試しモード

# 有効化すると以下の変更が加わる:
# ダウンロードするデータサイズ 24GB -> 800MB
# BPB評価のインターバルステップ数 250 -> 10
# COREベンチマークのインターバルステップ数 2000 -> 10 
# サンプル生成のインターバルステップ数 2000 -> 10

OTAMESHI_MODE = True

In [ ]:
# RustBPEのビルドとインストール

%pip install maturin
import os

if not os.path.exists("nanochat"):
    !git clone https://github.com/karpathy/nanochat

try:
    # Google Colabの場合
    from google.colab import userdata

    # ターミナルでRustをインストール
    # curl --proto '=https' --tlsv1.2 -sSf https://sh.rustup.rs | sh -s -- -y && . "$HOME/.cargo/env"

    # ターミナルでRustBPEのビルドとインストール
    # maturin build --release --manifest-path nanochat/rustbpe/Cargo.toml

    # ターミナルで生成されたwhlファイルをインストール
    # pip install nanochat/rustbpe/target/wheels/*.whl

    if not os.path.exists("nanochat/rustbpe/target"):
        raise FileNotFoundError("rustbpeのビルドとインストールをターミナルで実行してください。")

except ImportError:
    # ローカル環境の場合
    !maturin develop --release --manifest-path nanochat/rustbpe/Cargo.toml

import rustbpe

In [ ]:
# ログ設定

import logging as logging

if os.path.exists('debug.log'):
    os.remove('debug.log')

def custom_format(record):
    match record.levelno:
        case logging.DEBUG:
            level = '🟦'
        case logging.INFO:
            level = '🟩'
        case logging.WARNING:
            level = '🟨'
        case logging.ERROR:
            level = '🟥'
        case logging.CRITICAL:
            level = '🛑'
    return f"{level} {record.getMessage()}"

logger = logging.getLogger()

for handler in logger.handlers:
    logger.removeHandler(handler)

formatter = logging.Formatter()
formatter.format = custom_format

file_handler = logging.FileHandler('debug.log')
file_handler.setFormatter(formatter)
logger.addHandler(file_handler)

stream_handler = logging.StreamHandler()
stream_handler.setFormatter(formatter)
logger.addHandler(stream_handler)

logger.setLevel(logging.DEBUG)
logger.debug("ログを初期化")

In [ ]:
# ベースディレクトリを設定

def get_base_dir():
    """
    ベースディレクトリのパスを取得する
    デフォルトは、~/.cache/nanochat
    NANOCHAT_BASE_DIR環境変数で上書き可能

    Returns:
        str: ベースディレクトリのパス
    """
    if os.environ.get("NANOCHAT_BASE_DIR"):
        nanochat_dir = os.environ.get("NANOCHAT_BASE_DIR")
    else:
        home_dir = os.path.expanduser("~")
        cache_dir = os.path.join(home_dir, ".cache")
        nanochat_dir = os.path.join(cache_dir, "nanochat")
    os.makedirs(nanochat_dir, exist_ok=True)
    return nanochat_dir

logger.debug(f"ベースディレクトリ: {get_base_dir()}")

## 事前学習データセットの構築

事前学習データセットは、トークナイザーの訓練とモデルの事前学習に使用するデータセット

[karpathy/fineweb-edu-100b-shuffle][2]を使用:

- [FineWeb-EDUデータセット][1]を、シャッフルしシャード（断片ファイル）に再パッケージ
- 各シャードは25万文字のparquetファイルで、約100MB
- 合計シャード数は1822
- depth=20のモデルを訓練するために必要なシャード数は240で、約24GB
- 必要なシャード数は[Chinchillaのスケーリング則][3]で求める（訓練に必要なトークン数 = 20 * モデルのパラメータ数）
    1. 20層のモデルのパラメータ数が561M（5億6100万パラメータ）とすると、20 * 561M = 11.2B（11.2億）トークンが必要
    1. トークナイザーの性能が4.8文字/トークンとすると、11.2 * 4.8 = 54B（540億）文字が必要
    1. シャードあたり250M（2億5000万）文字であるため、54B / 250M = 216シャードが最低必要
    1. キリよく240シャードが必要

[1]: https://huggingface.co/datasets/HuggingFaceFW/fineweb-edu
[2]: https://huggingface.co/datasets/karpathy/fineweb-edu-100b-shuffle
[3]: https://arxiv.org/abs/2203.15556

In [ ]:
import os
import argparse
import time
import requests
import pyarrow.parquet as pq
from multiprocessing import Pool

### ダウンロード設定

In [ ]:
# データセットのURL
# 日本語のデータセットも可能
BASE_URL = "https://huggingface.co/datasets/karpathy/fineweb-edu-100b-shuffle/resolve/main"
logger.debug(f"データセットURL {BASE_URL=}")

# データセットの最大シャードインデックス
# fineweb-edu-100b-shuffleの場合、1823個のシャードがあり、インデックスは0から1822まで
MAX_SHARD = 1822
logger.debug(f"データセットの最大シャードインデックス {MAX_SHARD=}")

# ダウンロードするシャードの数
num_files = 8 if OTAMESHI_MODE else 240
logger.debug(f"ダウンロードするシャード数 {num_files=}")

# ダウンロードに使用するワーカー数（プロセス数）
num_workers = 4
logger.debug(f"ダウンロードに使用するワーカー数 {num_workers=}")

# データセットの保存ディレクトリの作成
base_dir = get_base_dir()
DATA_DIR = os.path.join(base_dir, "base_data")
os.makedirs(DATA_DIR, exist_ok=True)
logger.debug(f"データセット保存ディレクトリ {DATA_DIR=}")

### ヘルパー関数

複数のシャードを並列してダウンロードするための関数を作成

In [ ]:
def index_to_filename(index):
    """
    シャードのインデックスをファイル名に変換する

    Args:
        index (int): シャードのインデックス
    Returns:
        str: ファイル名
    """
    return f"shard_{index:05d}.parquet"

index_to_filename(0)

In [ ]:
def download_single_file(index):
    """
    単一のシャードをダウンロードする

    Args:
        index (int): ダウンロードするシャードのインデックス
    Returns:
        bool: ダウンロードが成功したかどうか
    """
    logger.debug(f"シャードのダウンロードを開始 {index=}")

    # 1) 初期化

    # ダウンロード先のファイルパスを作成
    filename = index_to_filename(index)
    filepath = os.path.join(DATA_DIR, filename)
    logger.debug(f"保存先 {filepath=}")

    # 既にファイルが存在する場合はスキップ
    if os.path.exists(filepath):
        logger.debug(f"ダウンロード済みのためスキップ {index=}")
        return True

    # ダウンロードURLを作成
    url = f"{BASE_URL}/{filename}"
    logger.debug(f"ダウンロードURL {url=}")

    # 2) ダウンロード

    # ダウンロード（最大5回のリトライ）
    max_attempts = 5
    for attempt in range(1, max_attempts + 1):
        try:
            # ストリームモードでダウンロード
            logger.debug(f"ダウンロード試行 {attempt}/{max_attempts} {index=}")
            response = requests.get(url, stream=True, timeout=30)
            response.raise_for_status()

            temp_path = filepath + f".tmp"
            logger.debug(f"一時ファイルに書き込み {temp_path=}")
            with open(temp_path, 'wb') as f:
                for chunk in response.iter_content(chunk_size=1024 * 1024):  # 1MB chunks
                    if chunk:
                        f.write(chunk)

            os.rename(temp_path, filepath)
            logger.debug(f"ダウンロード完了 {index=}")
            return True

        except (requests.RequestException, IOError) as e:
            logger.warning(f"ダウンロードエラー {e} {index=} {attempt=}")

            # 不完全なファイルを削除
            for path in [filepath + f".tmp", filepath]:
                if os.path.exists(path):
                    try:
                        os.remove(path)
                    except:
                        pass

            # バックオフ
            # 2^0, 2^1, 2^2, ...秒待機してリトライ
            if attempt < max_attempts:
                wait_time = 2 ** attempt
                logger.debug(f"バックオフ: {wait_time} 秒待機中")
                time.sleep(wait_time)
            else:
                logger.debug(f"シャードインデックス {index} のダウンロードに失敗")
                return False

    return False

download_single_file(0)

### 並列ダウンロード

In [ ]:
# 1) シャードインデックスのリストを作成

num = MAX_SHARD + 1 if num_files == -1 else min(num_files, MAX_SHARD + 1)
ids_to_download = list(range(num))
logger.debug(f"ダウンロードするシャードインデックス: {ids_to_download=}")

# 2) 並列処理でダウンロードを実行

with Pool(processes=num_workers) as pool:
    results = pool.map(download_single_file, ids_to_download)

# 3) 結果を表示

successful = sum(1 for success in results if success)
success_rate = successful / len(ids_to_download)
logger.debug(f"並列ダウンロード完了 {success_rate=}")

## トークナイザーの実装

rustbpeをラップしたトークナイザークラスRustBPETokenizerを実装

In [ ]:
import pickle
import rustbpe
import tiktoken
import os
import copy
from functools import lru_cache

#### 設定

In [ ]:
# 特殊トークン

SPECIAL_TOKENS = [
    # 事前学習で使用
    "<|bos|>", # 文の開始
    # ファインチューニング時に使用
    "<|user_start|>", # ユーザーメッセージ
    "<|user_end|>",
    "<|assistant_start|>", # アシスタントメッセージ
    "<|assistant_end|>",
    "<|python_start|>", # アシスタントがPython REPLツールを呼び出す
    "<|python_end|>",
    "<|output_start|>", # Python REPLがアシスタントに出力を返す
    "<|output_end|>",
]

logger.debug(f"特殊トークン数 {len(SPECIAL_TOKENS)=}")

In [ ]:
# 事前トークン化の正規表現を設定

# GPT-4と同じ
SPLIT_PATTERN = r"""'(?i:[sdmt]|ll|ve|re)|[^\r\n\p{L}\p{N}]?+\p{L}+|\p{N}{1,2}| ?[^\s\p{L}\p{N}]++[\r\n]*|\s*[\r\n]|\s+(?!\S)|\s+"""

logger.debug(f"事前トークン化の正規表現: {SPLIT_PATTERN=}")

### RustBPETokenizer

In [ ]:
class RustBPETokenizer:
    """
    rustbpeのラッパークラス
    訓練時はrustbpeを使用し、推論時はtiktokenを使用する
    """

    def __init__(self, enc, bos_token):
        logger.debug(f"RustBPETokenizer初期化開始 {enc=} {bos_token=}")

        # tiktokenのEncodingオブジェクト
        self.enc = enc 

        # BOSトークンIDをキャッシュ
        self.bos_token_id = self.encode_special(bos_token)
        logger.debug(f"RustBPETokenizer初期化完了 {self.bos_token_id=}")

    @classmethod # クラスメソッドとして定義
    def train_from_iterator(cls, text_iterator, vocab_size):
        """
        rustbpeでトークナイザーを訓練し、tiktokenのエンコーダーを構築する

        Args:
            text_iterator (iterable): テキストのイテレータ
            vocab_size (int): 語彙数
        Returns:
            RustBPETokenizer: 訓練済みのRustBPETokenizerオブジェクト
        """

        logger.debug(f"RustBPETokenizerの訓練開始 {vocab_size=}")

        # 1) rustbpeを訓練

        # rustbpeのTokenizerオブジェクトを作成
        tokenizer = rustbpe.Tokenizer()

        # 特殊トークンは後で__init__で挿入されるため、ここでは訓練しない
        vocab_size_no_special = vocab_size - len(SPECIAL_TOKENS)

        assert vocab_size_no_special >= 256, f"vocab_size_no_special must be at least 256, got {vocab_size_no_special}"

        # トークナイザーを訓練
        tokenizer.train_from_iterator(text_iterator, vocab_size_no_special, pattern=SPLIT_PATTERN)

        # 2) tiktokenのエンコーダーを構築

        # 事前トークン化の正規表現パターン
        pattern = tokenizer.get_pattern()

        # tiktokenに対応したマージルールを作成
        # {バイト列: マージの優先順位ランク}
        mergeable_ranks_list = tokenizer.get_mergeable_ranks()
        mergeable_ranks = {bytes(k): v for k, v in mergeable_ranks_list}

        # tiktokenに対応した特殊トークンの辞書を作成
        # {トークン名: トークンID}
        tokens_offset = len(mergeable_ranks)
        special_tokens = {name: tokens_offset + i for i, name in enumerate(SPECIAL_TOKENS)}

        # tiktokenのエンコーダーを構築
        enc = tiktoken.Encoding(
            name="rustbpe",
            pat_str=pattern,
            mergeable_ranks=mergeable_ranks,
            special_tokens=special_tokens,
        )

        logger.debug(f"RustBPETokenizerの訓練完了")

        # RustBPETokenizerオブジェクトを返す
        # clsでこのクラスのコンストラクタが呼ばれる
        return cls(enc, "<|bos|>")

    @classmethod
    def from_directory(cls, tokenizer_dir):
        """
        訓練済みのRustBPETokenizerをディレクトリから読み込む

        Args:
            tokenizer_dir (str): トークナイザーの保存ディレクトリ
        Returns:
            RustBPETokenizer: 読み込んだRustBPETokenizerオブジェクト
        """

        logger.debug(f"RustBPETokenizerをディレクトリから読み込み開始 {tokenizer_dir=}")

        pickle_path = os.path.join(tokenizer_dir, "tokenizer.pkl")
        logger.debug(f"トークナイザーファイルパス {pickle_path=}")

        with open(pickle_path, "rb") as f:
            enc = pickle.load(f)

        logger.debug(f"RustBPETokenizerをディレクトリから読み込み完了 {enc=}")
        return cls(enc, "<|bos|>")

    @classmethod
    def from_pretrained(cls, tiktoken_name):
        """
        学習済みのtiktokenトークナイザーを読み込む

        Args:
            tiktoken_name (str): tiktokenのエンコーディング名
        Returns:
            RustBPETokenizer: 読み込んだRustBPETokenizerオブジェクト
        """
        logger.debug(f"学習済みのtiktokenトークナイザーを読み込み開始 {tiktoken_name=}")

        # https://github.com/openai/tiktoken/blob/eedc8563/tiktoken_ext/openai_public.py
        enc = tiktoken.get_encoding(tiktoken_name)

        # nanochatでは<|bos|>を使用するが、tiktokenのgpt2などでは<|endoftext|>が使用されているため

        logger.debug(f"学習済みのtiktokenトークナイザーを読み込み完了 {enc=}")
        return cls(enc, "<|endoftext|>")

    def get_vocab_size(self):
        return self.enc.n_vocab

    def get_special_tokens(self):
        return self.enc.special_tokens_set

    def id_to_token(self, id):
        return self.enc.decode([id])

    @lru_cache(maxsize=32)
    def encode_special(self, text):
        return self.enc.encode_single_token(text)

    def get_bos_token_id(self):
        return self.bos_token_id

    def encode(self, text, prepend=None, append=None, num_threads=8):
        """
        テキストをトークンIDにエンコードする

        Args:
            text (str or list[str]): エンコードするテキストまたはテキストのリスト
            prepend (str or int, optional): 先頭に追加する特殊トークン（文字列またはトークンID）
            append (str or int, optional): 末尾に追加する特殊トークン（文字列またはトークンID）
            num_threads (int, optional): バッチエンコード時のスレッド数（デフォルト: 8）
        Returns:
            list[int] or list[list[int]]: トークンIDのリストまたはトークンIDのリストのリスト
        """
        logger.debug(f"エンコード開始 {len(text)=} {prepend=} {append=} {num_threads=}")

        # 1) 特殊トークンの設定

        # 出力の先頭に付ける特殊トークンIDを取得
        # BOSトークン
        if prepend is not None:
            prepend_id = prepend if isinstance(prepend, int) else self.encode_special(prepend)
            logger.debug(f"{prepend_id=}") # 65527

        # 出力の末尾に付ける特殊トークンIDを取得
        # なし
        if append is not None:
            append_id = append if isinstance(append, int) else self.encode_special(append)
            logger.debug(f"{append_id=}") 

        # 2) エンコード

        # 入力が文字列の場合
        if isinstance(text, str):

            # tiktokenでエンコード
            ids = self.enc.encode_ordinary(text)

            if prepend is not None:
                # 先頭に特殊トークンを追加
                ids.insert(0, prepend_id)

            if append is not None:
                # 末尾に特殊トークンを追加
                ids.append(append_id)

        # 入力が文字列のリストの場合
        elif isinstance(text, list):

            # tiktokenでバッチエンコード
            ids = self.enc.encode_ordinary_batch(text, num_threads=num_threads)

            if prepend is not None:
                # 先頭に特殊トークンを追加
                for ids_row in ids:
                    ids_row.insert(0, prepend_id)

            if append is not None:
                # 末尾に特殊トークンを追加
                for ids_row in ids:
                    ids_row.append(append_id)
        else:
            raise ValueError(f"Invalid input type: {type(text)}")

        logger.debug(f"エンコード完了 {len(ids)=}")
        return ids

    def __call__(self, *args, **kwargs):
        return self.encode(*args, **kwargs)

    def decode(self, ids):
        """
        トークンIDのリストをデコードしてテキストに変換する

        Args:
            ids (list[int]): トークンIDのリスト
        Returns:
            str: デコードしたテキスト
        """
        logger.debug(f"デコード開始 {len(ids)=}")

        # tiktokenでデコード
        res = self.enc.decode(ids)

        logger.debug(f"デコード完了 {len(res)=}")
        return res

    def save(self, tokenizer_dir):
        """
        訓練後のトークナイザーをディレクトリに保存する

        Args:
            tokenizer_dir (str): トークナイザーの保存ディレクトリ
        """
        logger.debug(f"トークナイザーの保存開始 {tokenizer_dir=}")

        # 1) ディレクトリを作成

        os.makedirs(tokenizer_dir, exist_ok=True)
        pickle_path = os.path.join(tokenizer_dir, "tokenizer.pkl")
        logger.debug(f"トークナイザーファイルパス {pickle_path=}")

        # 2) pickleで保存

        with open(pickle_path, "wb") as f:
            pickle.dump(self.enc, f)

        logger.debug(f"トークナイザーの保存完了 {pickle_path=}")

    def render_conversation(self, conversation, max_tokens=2048):
        """
        チャット形式のデータをトークン化する
        事前学習では使わない

        Args:
            conversation: dict チャット会話データ
            max_tokens: int 最大トークン数
        Returns:
            ids: list[int] トークンIDのリスト
            mask: list[int] 同じ長さのマスクリスト、mask=1はアシスタントが学習すべきトークン
        """
        logger.debug(f"会話のレンダリング開始 {conversation=} {max_tokens=}")

        # ids, masks that we will return and a helper function to help build them up.
        ids, mask = [], []

        def add_tokens(token_ids, mask_val):
            if isinstance(token_ids, int):
                token_ids = [token_ids]
            ids.extend(token_ids)
            mask.extend([mask_val] * len(token_ids))

        # sometimes the first message is a system message...
        # => just merge it with the second (user) message
        if conversation["messages"][0]["role"] == "system":
            # some conversation surgery is necessary here for now...
            conversation = copy.deepcopy(conversation) # avoid mutating the original
            messages = conversation["messages"]
            assert messages[1]["role"] == "user", "System message must be followed by a user message"
            messages[1]["content"] = messages[0]["content"] + "\n\n" + messages[1]["content"]
            messages = messages[1:]
        else:
            messages = conversation["messages"]

        assert len(messages) >= 1, f"Conversation has less than 1 message: {messages}"

        # fetch all the special tokens we need
        bos = self.get_bos_token_id()
        user_start, user_end = self.encode_special("<|user_start|>"), self.encode_special("<|user_end|>")
        assistant_start, assistant_end = self.encode_special("<|assistant_start|>"), self.encode_special("<|assistant_end|>")
        python_start, python_end = self.encode_special("<|python_start|>"), self.encode_special("<|python_end|>")
        output_start, output_end = self.encode_special("<|output_start|>"), self.encode_special("<|output_end|>")

        # 開始トークンを追加
        add_tokens(bos, 0)

        # メッセージを順に処理
        for i, message in enumerate(messages):

            # some sanity checking here around assumptions, to prevent footguns
            must_be_from = "user" if i % 2 == 0 else "assistant"
            assert message["role"] == must_be_from, f"Message {i} is from {message['role']} but should be from {must_be_from}"

            # content can be either a simple string or a list of parts (e.g. containing tool calls)
            content = message["content"]

            # ユーザーメッセージの場合
            if message["role"] == "user":
                assert isinstance(content, str), "User messages are simply expected to be strings"
                value_ids = self.encode(content)

                # 損失計算に含めない
                add_tokens(user_start, 0)
                add_tokens(value_ids, 0)
                add_tokens(user_end, 0)

            # アシスタントメッセージの場合
            elif message["role"] == "assistant":

                # アシスタントの開始トークンを追加
                add_tokens(assistant_start, 0)

                if isinstance(content, str):
                    # テキストをトークン化
                    value_ids = self.encode(content)

                    # 損失計算に含める
                    add_tokens(value_ids, 1)

                elif isinstance(content, list):
                    for part in content:
                        # テキストをトークン化
                        value_ids = self.encode(part["text"])

                        # 通常のテキストの場合
                        if part["type"] == "text":

                            # 損失計算に含める
                            add_tokens(value_ids, 1)

                        # Pythonのツール呼び出しの場合
                        elif part["type"] == "python":
                            # ツールの開始と終了トークンを追加
                            add_tokens(python_start, 1)
                            add_tokens(value_ids, 1)
                            add_tokens(python_end, 1)

                        # Pythonの出力の場合
                        elif part["type"] == "python_output":
                            # ツールの出力の開始と終了トークンを追加
                            # 推論時はPythonの出力を用いるため損失計算に含めない
                            add_tokens(output_start, 0)
                            add_tokens(value_ids, 0)
                            add_tokens(output_end, 0)
                        else:
                            raise ValueError(f"Unknown part type: {part['type']}")
                else:
                    raise ValueError(f"Unknown content type: {type(content)}")
                add_tokens(assistant_end, 1)

        # 最大トークン数に切り詰める
        ids = ids[:max_tokens]
        mask = mask[:max_tokens]

        logger.debug(f"会話のレンダリング完了 {ids=} {mask=}")
        return ids, mask

    def visualize_tokenization(self, ids, mask, with_token_id=False):
        """
        render_conversationの結果の可視化
        今回は使わない
        """
        RED = '\033[91m'
        GREEN = '\033[92m'
        RESET = '\033[0m'
        GRAY = '\033[90m'
        tokens = []
        for i, (token_id, mask_val) in enumerate(zip(ids, mask)):
            token_str = self.decode([token_id])
            color = GREEN if mask_val == 1 else RED
            tokens.append(f"{color}{token_str}{RESET}")
            if with_token_id:
                tokens.append(f"{GRAY}({token_id}){RESET}")
        return '|'.join(tokens)

    def render_for_completion(self, conversation):
        """
        Assistantの補完を促すために会話をレンダリングする
        今回は使わない
        """
        # We have some surgery to do: we need to pop the last message (of the Assistant)
        conversation = copy.deepcopy(conversation) # avoid mutating the original
        messages = conversation["messages"]
        assert messages[-1]["role"] == "assistant", "Last message must be from the Assistant"
        messages.pop() # remove the last message (of the Assistant) inplace

        # Now tokenize the conversation
        ids, mask = self.render_conversation(conversation)

        # Finally, to prime the Assistant for a completion, append the Assistant start token
        assistant_start = self.encode_special("<|assistant_start|>")
        ids.append(assistant_start)
        return ids

## トークナイザーの訓練

In [ ]:
import os
import time
import argparse
import torch

#### 設定

In [ ]:
# 訓練に使用する最大文字数
# デフォルトは10B（100億）文字だが、多いので2B（20億）文字に設定
max_chars = 2_000_000_000
logger.info(f"トークナイザーの訓練に使用する最大文字数: {max_chars=:,}")

# 1ドキュメントあたりの最大文字数
# デフォルトは10,000文字
doc_cap = 10_000
logger.info(f"1ドキュメントあたりの最大文字数: {doc_cap=:,}")

# トークナイザーの語彙数
# デフォルトは65,536（2の16乗）
vocab_size = 65_536
logger.info(f"トークナイザーの語彙数: {vocab_size=:,}")

### ヘルパー関数

parquet形式のシャードからテキストを読み込む関数を作成

In [ ]:
def list_parquet_files(data_dir=None):
    """
    データディレクトリ内の全てのparquetファイルのフルパスを返す
    Args:
        data_dir (str, optional): データディレクトリのパス。Noneの場合、デフォルトのDATA_DIRを使用する。
    Returns:
        List[str]: parquetファイルのフルパスのリスト
    """
    # データディレクトリのパスを設定
    data_dir = DATA_DIR if data_dir is None else data_dir

    # ディレクトリ内のparquetファイルをリストアップ
    parquet_files = sorted([
        f for f in os.listdir(data_dir)
        if f.endswith('.parquet') and not f.endswith('.tmp')
    ])

    # フルパスに変換
    parquet_paths = [os.path.join(data_dir, f) for f in parquet_files]

    return parquet_paths

list_parquet_files()[:5]

In [ ]:
def parquets_iter_batched(split, start=0, step=1):
    """
    データセットをバッチで読み込むためのイテレータを返す

    splitは、"train"または"val"を指定
    "train"の場合、最初のN-1ファイルを使用
    "val"の場合、最後のファイルを使用
    startとstepは、分散データ処理（Distributed Data Processing）でスキップする際に使用

    Args:
        split (str): "train"または"val"
        start (int, optional): 開始インデックス。デフォルトは0。
        step (int, optional): ステップサイズ。デフォルトは1。
    Yields:
        List[str]: 各バッチのテキストデータのリスト
    """
    logger.debug(f"データセットを読み込み開始 {split=} {start=} {step=}")

    # 1) 使用するシャードのパスのリストを作成

    assert split in ["train", "val"], "split must be 'train' or 'val'"

    # すべてのparquetファイルのパスを取得
    parquet_paths = list_parquet_files()
    logger.debug(f"全てのparquetファイル {parquet_paths=}")

    # splitに応じてファイルを選択
    parquet_paths = parquet_paths[:-1] if split == "train" else parquet_paths[-1:]
    logger.debug(f"使用するparquetファイル: {parquet_paths=}")

    # 2) 各ファイルの行グループごとにジェネレーターで出力する

    for filepath in parquet_paths:
        # Parquetファイルを読み込む
        pf = pq.ParquetFile(filepath)
        logger.debug(f"{filepath=} {pf.num_row_groups=}")

        # ファイル内の行グループ（row groups）でループ
        # 行グループは、数千から数万行のデータのチャンク
        for rg_idx in range(start, pf.num_row_groups, step):

            # 行グループを読み込む
            # pyarrow.lib.Table, 1024行
            rg = pf.read_row_group(rg_idx)
            logger.debug(f"{type(rg)=} {len(rg)=:,}")

            # テキストカラムをリストとして取得してyield
            # list, 1024行, 4278文字
            texts = rg.column('text').to_pylist()
            logger.debug(f"{type(texts)=} {len(texts)=:,} {len(texts[0])=:,}")

            # 1グループ分のテキストを返す
            yield texts

text = parquets_iter_batched("train")
first_row_group = next(text)
first_row_group[0][:50]

In [ ]:
def text_iterator():
    """
    トークナイザーの訓練にテキストデータを1ずつ供給するイテレータ

    Returns:
        Iterator[str]: テキストデータのイテレータ

    1) Flatten the batches into a single iterator
    2) Crop every document to args.doc_cap characters
    3) Break when we've seen args.max_chars characters
    """
    # 文字数カウンタを初期化
    nchars = 0

    # 行グループ（バッチ）ごとにループ
    for batch in parquets_iter_batched(split="train"):

        # バッチに含まれるドキュメントごとにループ
        for doc in batch:

            # ドキュメントは平均9,000文字程度
            doc_text = doc
            logger.debug(f"ドキュメントからテキストを取得 {len(doc_text)=:,}")

            # ドキュメントの文字数が10,000文字を超える場合、切り詰める
            if len(doc_text) > doc_cap:
                doc_text = doc_text[:doc_cap]
                logger.debug(f"ドキュメントを切り詰め {len(doc_text)=:,}")

            # 文字数カウンタを更新
            nchars += len(doc_text)
            logger.debug(f"累積文字数を更新 {nchars=}")

            # ドキュメントのテキストをイテレーターで出力
            yield doc_text

            # 最大文字数に達した場合、終了
            if nchars > max_chars:
                logger.debug(f"最大文字数に達したため終了 {nchars=}")
                return

text_iter = text_iterator()
first_doc = next(text_iter)
len(first_doc), first_doc[:50]

### 訓練

In [ ]:
# トークナイザーの保存ディレクトリを設定
base_dir = get_base_dir()
tokenizer_dir = os.path.join(base_dir, "tokenizer")
logger.debug(f"トークナイザーの保存ディレクトリ: {tokenizer_dir}")

# 訓練済みでない場合、トークナイザーを訓練
if not os.path.exists(tokenizer_dir):
    logger.debug("トークナイザーの訓練を開始")

    logger.setLevel(logging.ERROR)
    t0 = time.time()

    # イテレータを初期化
    text_iter = text_iterator()

    # トークナイザーを訓練
    tokenizer = RustBPETokenizer.train_from_iterator(text_iter, vocab_size)

    t1 = time.time()

    # RTX5090の場合、26秒程度
    train_time = t1 - t0

    logger.setLevel(logging.DEBUG)
    logger.debug(f"トークナイザーの訓練完了 {train_time=}秒")

    # トークナイザーを保存
    tokenizer.save(tokenizer_dir)

# 訓練済みの場合、トークナイザーを読み込み
else:
    logger.debug("既存のトークナイザーを読み込み")
    tokenizer = RustBPETokenizer.from_directory(tokenizer_dir)

In [ ]:
# トークナイザーを検証

test_text = """Hello world! This is a test.
Numbers: 123, 4567, 89
Contractions: I'm, you're, it's
Special chars: @#$%^&*()
Unicode: こんにちは世界 🌍"""

encoded = tokenizer.encode(test_text)
logger.debug(f"エンコード結果: {encoded}")

decoded = tokenizer.decode(encoded)
logger.debug(f"デコード結果: {decoded}")
assert decoded == test_text

### トークンとバイト数の対応リストを作成

モデルの訓練の損失計算を、バイト数ごとの平均損失にする（BPB評価）ための対応リストを作成

異なるトークナイザーを使用した場合でもモデルの性能を公平に比較できる

In [ ]:
vocab_size = tokenizer.get_vocab_size()
logger.debug(f"トークナイザーの語彙数: {vocab_size=}")

special_set = set(tokenizer.get_special_tokens())
logger.debug(f"特殊トークン数: {len(special_set)=}")

token_bytes_path = os.path.join(tokenizer_dir, "token_bytes.pt")
logger.debug(f"トークンとバイト数の対応リストの保存パス: {token_bytes_path=}")

# 対応リストを作成していない場合
if not os.path.exists(token_bytes_path):
    logger.setLevel(logging.ERROR)

    # トークンIDを文字列にデコード
    token_strings = [tokenizer.decode([token_id]) for token_id in range(vocab_size)]
    token_bytes = []

    # 全てのトークンIDでループ
    for token_id in range(vocab_size):

        # トークンIDに対応する文字列を取得
        token_str = token_strings[token_id]

        # 特殊トークンの場合
        if token_str in special_set:
            # 特殊トークンは損失計算対象外なため、0バイトとして対応リストに追加
            token_bytes.append(0)
        else:
            # UTF-8のバイト列に変換し、そのバイト数を対応リストに追加
            id_bytes = len(token_str.encode("utf-8"))
            token_bytes.append(id_bytes)

    # 対応リストをテンソルに変換
    token_bytes = torch.tensor(token_bytes, dtype=torch.int32, device='cpu')

    # 対応リストを保存
    with open(token_bytes_path, "wb") as f:
        torch.save(token_bytes, f)

    logger.setLevel(logging.DEBUG)

# 対応リストを作成済みの場合
else:
    # 対応リストを読み込み
    token_bytes = torch.load(token_bytes_path)

# 統計を表示
token_bytes_nonzero = (token_bytes[token_bytes > 0]).to(dtype=torch.float32)
logger.debug(f"トークンあたりの最小バイト数 {int(token_bytes_nonzero.min().item())}")
logger.debug(f"トークンあたりの最大バイト数 {int(token_bytes_nonzero.max().item())}")
logger.debug(f"トークンあたりのバイト数の平均 {token_bytes_nonzero.mean().item()}")
logger.debug(f"トークンあたりのバイト数の標準偏差 {token_bytes_nonzero.std().item()}")

# 検証
token_id = 5000
token_str = tokenizer.decode([token_id])
num_bytes = token_bytes[token_id].item()
logger.debug(f"Token ID: {token_id}, Token: '{token_str}', Bytes: {num_bytes}")

## トークナイザーの評価

#### 評価用のテキストを作成

In [ ]:
# ニュース記事テキスト

news_text = r"""
(Washington, D.C., July 9, 2025)- Yesterday, Mexico’s National Service of Agro-Alimentary Health, Safety, and Quality (SENASICA) reported a new case of New World Screwworm (NWS) in Ixhuatlan de Madero, Veracruz in Mexico, which is approximately 160 miles northward of the current sterile fly dispersal grid, on the eastern side of the country and 370 miles south of the U.S./Mexico border. This new northward detection comes approximately two months after northern detections were reported in Oaxaca and Veracruz, less than 700 miles away from the U.S. border, which triggered the closure of our ports to Mexican cattle, bison, and horses on May 11, 2025.

While USDA announced a risk-based phased port re-opening strategy for cattle, bison, and equine from Mexico beginning as early as July 7, 2025, this newly reported NWS case raises significant concern about the previously reported information shared by Mexican officials and severely compromises the outlined port reopening schedule of five ports from July 7-September 15. Therefore, in order to protect American livestock and our nation’s food supply, Secretary Rollins has ordered the closure of livestock trade through southern ports of entry effective immediately.

“The United States has promised to be vigilant — and after detecting this new NWS case, we are pausing the planned port reopening’s to further quarantine and target this deadly pest in Mexico. We must see additional progress combatting NWS in Veracruz and other nearby Mexican states in order to reopen livestock ports along the Southern border,” said U.S. Secretary of Agriculture Brooke L. Rollins. “Thanks to the aggressive monitoring by USDA staff in the U.S. and in Mexico, we have been able to take quick and decisive action to respond to the spread of this deadly pest.”
""".strip()

In [ ]:
# 非英語テキスト

korean_text = r"""
정직한 사실 위에, 공정한 시선을 더하다
Herald Korea Times

헤럴드코리아타임즈는 정치, 경제, 사회, 문화 등 한국 사회 전반의 주요 이슈를 심도 있게 다루는 종합 온라인 신문사입니다.

우리는 단순히 뉴스를 전달하는 것이 아니라, 사실(Fact)에 기반한 양측의 시각을 균형 있게 조명하며, 독자 여러분이 스스로 판단할 수 있는 ‘정보의 균형’을 제공합니다.

한국 언론의 오랜 문제로 지적되어 온 정치적 편향, 이념적 왜곡에서 벗어나
오직 정직함과 공정함을 원칙으로 삼는 언론을 지향합니다.
어느 한쪽의 주장만을 확대하거나 감추지 않고,
**모든 쟁점에 대해 ‘무엇이 쟁점인지’, ‘누가 무엇을 주장하는지’, ‘사실은 무엇인지’**를 명확히 전달하는 데 집중합니다.
""".strip()

In [ ]:
# コードスニペット

code_text = r"""
class BasicTokenizer(Tokenizer):

    def __init__(self):
        super().__init__()

    def train(self, text, vocab_size, verbose=False):
        assert vocab_size >= 256
        num_merges = vocab_size - 256

        # input text preprocessing
        text_bytes = text.encode("utf-8") # raw bytes
        ids = list(text_bytes) # list of integers in range 0..255

        # iteratively merge the most common pairs to create new tokens
        merges = {} # (int, int) -> int
        vocab = {idx: bytes([idx]) for idx in range(256)} # int -> bytes
        for i in range(num_merges):
            # count up the number of times every consecutive pair appears
            stats = get_stats(ids)
            # find the pair with the highest count
            pair = max(stats, key=stats.get)
            # mint a new token: assign it the next available id
            idx = 256 + i
            # replace all occurrences of pair in ids with idx
            ids = merge(ids, pair, idx)
            # save the merge
            merges[pair] = idx
            vocab[idx] = vocab[pair[0]] + vocab[pair[1]]
            # prints
            if verbose:
                print(f"merge {i+1}/{num_merges}: {pair} -> {idx} ({vocab[idx]}) had {stats[pair]} occurrences")
""".strip()

In [ ]:
# 数学テキスト

math_text = r"""
\documentclass[12pt]{article}
\usepackage{amsmath,amsthm,amssymb}
\usepackage[margin=1in]{geometry}

\newtheorem{theorem}{Theorem}
\newtheorem*{remark}{Remark}

\begin{document}

\begin{center}
{\Large A Cute Identity: The Sum of Cubes is a Square}
\end{center}

\begin{theorem}
For every integer $n \ge 1$,
\[
\sum_{k=1}^{n} k^{3} \;=\; \left(\frac{n(n+1)}{2}\right)^{2}.
\]
\end{theorem}

\begin{proof}[Proof 1 (Induction)]
Let $S(n) = \sum_{k=1}^{n} k^3$. For $n=1$, $S(1)=1=(1\cdot 2/2)^2$, so the base case holds.

Assume $S(n)=\big(\tfrac{n(n+1)}{2}\big)^2$ for some $n\ge 1$.
Then
\[
S(n+1)
= S(n) + (n+1)^3
= \left(\frac{n(n+1)}{2}\right)^2 + (n+1)^3.
\]
Factor out $(n+1)^2$:
\[
S(n+1)
= (n+1)^2\left( \frac{n^2}{4} + (n+1) \right)
= (n+1)^2\left( \frac{n^2 + 4n + 4}{4} \right)
= (n+1)^2\left( \frac{(n+2)^2}{4} \right).
\]
Thus
\[
S(n+1)=\left(\frac{(n+1)(n+2)}{2}\right)^2,
\]
which matches the claimed formula with $n$ replaced by $n+1$. By induction, the identity holds for all $n\ge 1$.
\end{proof}

\begin{proof}[Proof 2 (Algebraic telescoping)]
Recall the binomial identity
\[
(k+1)^4 - k^4 = 4k^3 + 6k^2 + 4k + 1.
\]
Summing both sides from $k=0$ to $n$ telescopes:
\[
(n+1)^4 - 0^4
= \sum_{k=0}^{n}\big(4k^3 + 6k^2 + 4k + 1\big)
= 4\sum_{k=1}^{n}k^3 + 6\sum_{k=1}^{n}k^2 + 4\sum_{k=1}^{n}k + (n+1).
\]
Using the standard sums
\[
\sum_{k=1}^{n}k = \frac{n(n+1)}{2}
\quad\text{and}\quad
\sum_{k=1}^{n}k^2 = \frac{n(n+1)(2n+1)}{6},
\]
solve for $\sum_{k=1}^{n}k^3$ to get
\[
\sum_{k=1}^{n}k^3 = \left(\frac{n(n+1)}{2}\right)^2.
\]
\end{proof}

\begin{remark}
Geometrically, the identity says: ``adding up $1^3,2^3,\dots,n^3$ builds a perfect square’’—namely the square of the $n$th triangular number. This is why one sometimes calls it the \emph{sum-of-cubes is a square} phenomenon.
\end{remark}

\end{document}
""".strip()

In [ ]:
# 科学テキスト

science_text = r"""
Photosynthesis is a photochemical energy transduction process in which light-harvesting pigment–protein complexes within the thylakoid membranes of oxygenic phototrophs absorb photons and initiate charge separation at the reaction center, driving the linear electron transport chain from water to NADP⁺ via photosystem II, the cytochrome b₆f complex, and photosystem I, concomitantly generating a trans-thylakoid proton motive force utilized by chloroplastic ATP synthase. The light-dependent reactions produce ATP and NADPH, which fuel the Calvin–Benson–Bassham cycle in the stroma, wherein ribulose-1,5-bisphosphate is carboxylated by ribulose-1,5-bisphosphate carboxylase/oxygenase (RuBisCO) to form 3-phosphoglycerate, subsequently reduced and regenerated through a series of enzymatic steps, enabling net assimilation of CO₂ into triose phosphates and ultimately carbohydrates. This process is tightly regulated by photoprotective mechanisms, redox feedback, and metabolite flux, representing a central biochemical pathway coupling solar energy capture to the biosphere’s primary productivity.
""".strip()

In [ ]:
# コーパスから訓練用と検証用のテキストを取得

train_docs = next(parquets_iter_batched(split="train"))
train_text = "\n".join(train_docs)
val_docs = next(parquets_iter_batched(split="val"))
val_text = "\n".join(val_docs)

In [ ]:
# 全てのテキストデータをリストに集約

all_text = [
    ("news", news_text),
    ("korean", korean_text),
    ("code", code_text),
    ("math", math_text),
    ("science", science_text),
    ("fwe-train", train_text),
    ("fwe-val", val_text),
]

#### ヘルパー関数

In [ ]:
def get_tokenizer():
    """
    RustBPETokenizerオブジェクトを返す

    Returns:
        RustBPETokenizer: トークナイザーオブジェクト
    """
    base_dir = get_base_dir()
    tokenizer_dir = os.path.join(base_dir, "tokenizer")
    # return HuggingFaceTokenizer.from_directory(tokenizer_dir)
    return RustBPETokenizer.from_directory(tokenizer_dir)

#### GPT-2とGPT-4のトークナイザーと比較

In [ ]:
logger.setLevel(logging.ERROR)

tokenizer_results = {}
vocab_sizes = {}

# 異なるトークナイザーでループ
for tokenizer_name in ["gpt2", "gpt4", "ours"]:

    # 1) トークナイザーを取得

    if tokenizer_name == "gpt2":
        tokenizer = RustBPETokenizer.from_pretrained("gpt2")
    elif tokenizer_name == "gpt4":
        tokenizer = RustBPETokenizer.from_pretrained("cl100k_base")
    else:
        tokenizer = get_tokenizer()

    # 語彙数を取得
    vocab_sizes[tokenizer_name] = tokenizer.get_vocab_size()

    # 結果を初期化
    tokenizer_results[tokenizer_name] = {}

    # 2) 検証

    for name, text in all_text:

        # エンコードとデコードを行い、完全一致を確認
        encoded = tokenizer.encode(text)
        decoded = tokenizer.decode(encoded)
        assert decoded == text

        # UTF-8のバイト列に変換
        encoded_bytes = text.encode('utf-8')

        # バイト数とトークン数の比率を計算
        ratio = len(encoded_bytes) / len(encoded)

        # 結果を保存
        tokenizer_results[tokenizer_name][name] = {
            'bytes': len(encoded_bytes),
            'tokens': len(encoded),
            'ratio': ratio
        }

logger.setLevel(logging.DEBUG)

In [ ]:
# 可視化

print(f"\n語彙数:")
print(f"GPT-2: {vocab_sizes['gpt2']}")
print(f"GPT-4: {vocab_sizes['gpt4']}")
print(f"Ours: {vocab_sizes['ours']}")

# ANSI color codes
GREEN = '\033[92m'
RED = '\033[91m'
RESET = '\033[0m'

def print_comparison(baseline_name, baseline_results, ours_results, all_text):
    print(f"\n{baseline_name}との比較:")
    print("=" * 95)
    print(f"{'Text Type':<10} {'Bytes':<8} {baseline_name:<15} {'Ours':<15} {'Relative':<12} {'Better':<10}")
    print(f"{'':10} {'':8} {'Tokens':<7} {'Ratio':<7} {'Tokens':<7} {'Ratio':<7} {'Diff %':<12}")
    print("-" * 95)

    for name, text in all_text:
        baseline_data = baseline_results[name]
        ours_data = ours_results[name]

        # Calculate relative difference (positive means ours is better, negative means worse)
        # Using tokens: fewer tokens is better, so we calculate (baseline_tokens - ours_tokens) / baseline_tokens
        relative_diff = ((baseline_data['tokens'] - ours_data['tokens']) / baseline_data['tokens']) * 100

        # Determine which has better compression (higher ratio = better)
        if baseline_data['ratio'] > ours_data['ratio']:
            baseline_color, ours_color = GREEN, RED
            better = baseline_name
            diff_color = RED
        elif ours_data['ratio'] > baseline_data['ratio']:
            baseline_color, ours_color = RED, GREEN
            better = "Ours"
            diff_color = GREEN
        else:
            baseline_color, ours_color = "", ""
            better = "Tie"
            diff_color = ""

        print(f"{name:<10} {baseline_data['bytes']:<8} "
              f"{baseline_color}{baseline_data['tokens']:<7}{RESET} "
              f"{baseline_color}{baseline_data['ratio']:<7.2f}{RESET} "
              f"{ours_color}{ours_data['tokens']:<7}{RESET} "
              f"{ours_color}{ours_data['ratio']:<7.2f}{RESET} "
              f"{diff_color}{relative_diff:+7.1f}%{RESET}     "
              f"{better:<10}")

# Print comparisons
print_comparison("GPT-2", tokenizer_results['gpt2'], tokenizer_results['ours'], all_text)
print_comparison("GPT-4", tokenizer_results['gpt4'], tokenizer_results['ours'], all_text)


## 最適化関数の実装

Transformerの最適化関数は[Muon（ミューオン, Momentum Orthgonalized by Newton-Schulz）][1]を使用

その他の最適化関数はAdamWを使用

[1]: https://arxiv.org/abs/2502.16982

In [ ]:
import torch
from torch import Tensor
import torch.distributed as dist

### ヘルパー関数

Muon最適化関数で使用する

In [ ]:
@torch.compile # PyTorch2.0のJITコンパイラを使用
def zeropower_via_newtonschulz5(G: Tensor, steps: int) -> Tensor:
    """
    ニュートン・シュルツ反復法で勾配の直行化を近似する

    行列Gの直交行列は、Gの特異値分解G = USV^Tに対してUV^Tで計算できる
    特異値分解（SVD）は計算コストが高いため近似手法を用いる
    原点における傾きを最大化するように選択された係数を持つ5次の反復法を採用

    Args: 
        G (Tensor): 直行化する行列、形状は(..., m, n)
        steps (int): 反復回数
    Returns:
        Tensor: 直交化された行列、形状は(..., m, n)
    """

    assert G.ndim >= 2

    # 5次反復法のための係数
    a, b, c = (3.4445, -4.7750,  2.0315)

    # 勾配をbfloat16にダウンキャスト
    X = G.bfloat16()

    # 安定化のため行列を横長にする
    if G.size(-2) > G.size(-1):
        X = X.mT

    # 行列のスペクトルノルム（最大の特異値）を1以下に正規化
    X = X / (X.norm(dim=(-2, -1), keepdim=True) + 1e-7)

    # ニュートン・シュワルツ反復法を適用
    for _ in range(steps):
        # A = X X^T
        A = X @ X.mT

        # B = b(X X^T) + c(X X^T)^2
        B = b * A + c * A @ A

        # X_k+1 = a X_k + (B) X_k 
        X = a * X + B @ X

    # 元の形状に戻す
    if G.size(-2) > G.size(-1):
        X = X.mT

    # 直交行列を返す
    return X

### Muon

In [ ]:
class Muon(torch.optim.Optimizer):
    """
    Muon最適化関数 https://kellerjordan.github.io/posts/muon/

    内部的に標準的なSGDモーメンタムを実行し、その後に直交化の後処理ステップを実行する
    直行化ステップでは、各2Dパラメータの更新が最も近い直交行列に置き換えられる
    各更新を効率的に直交化するために、ニュートン・シュルツ反復を使用する
    これにより、GPU上でbfloat16で安定して実行できる利点がある

    注意:
    - この最適化関数は、埋め込み層、最終全結合層、0次元・1次元パラメータには使用しない（AdamWなどを使用）
    - 4Dの畳み込みフィルタに使用する場合、最後の3つの次元をフラット化すると機能する

    Args:
        lr (float): 内部SGDで使用される学習率
        momentum (float): 内部SGDで使用されるモーメンタム
        nesterov (bool): 内部SGDでNesterovスタイルのモーメンタムを使用するかどうか（推奨）
        ns_steps (int): ニュートン・シュルツ反復のステップ数
    """

    def __init__(self, params, lr=0.02, momentum=0.95, nesterov=True, ns_steps=5):
        """
        Muon最適化関数の初期化

        Args:
            params (iterable): 最適化するパラメータ
            lr (float, optional): 内部SGDで使用される学習率
            momentum (float, optional): 内部SGDで使用されるモーメンタム
            nesterov (bool, optional): 内部SGDでNesterovスタイルのモーメンタムを使用するかどうか
            ns_steps (int, optional): ニュートン・シュルツ反復のステップ数
        """
        # 最適化関数のデフォルト設定
        defaults = dict(lr=lr, momentum=momentum, nesterov=nesterov, ns_steps=ns_steps)

        params: list[Tensor] = [*params]

        param_groups = []

        # 同じ要素数のパラメータをグループ化
        for size in {p.numel() for p in params}:
            group = dict(params=[p for p in params if p.numel() == size])
            param_groups.append(group)

        # 親クラスの初期化
        super().__init__(param_groups, defaults)

    @torch.no_grad()
    def step(self):
        """
        Muon最適化関数の1ステップの更新を実行
        """

        # 各パラメータグループでループ
        for group in self.param_groups:

            params: list[Tensor] = group["params"]

            # 各パラメータでループ
            for p in params:
                # 勾配を取得
                g = p.grad
                assert g is not None

                # モメンタムバッファを取得または初期化
                state = self.state[p]
                if "momentum_buffer" not in state:
                    state["momentum_buffer"] = torch.zeros_like(g)
                buf: Tensor = state["momentum_buffer"]

                # モメンタムバッファを最新の勾配で更新（指数移動平均）
                buf.lerp_(g, 1 - group["momentum"])

                # ネステロフ・モーメンタムを使用する場合、勾配を調整
                g = g.lerp_(buf, group["momentum"]) if group["nesterov"] else buf

                # 勾配を直交化
                g = zeropower_via_newtonschulz5(g, steps=group["ns_steps"])

                # パラメータを更新
                # -group["lr"]: 学習率を負にして減少方向に更新
                # max(1, p.size(-2) / p.size(-1))**0.5: 行列の形状に基づいてスケーリング
                p.add_(g, alpha=-group["lr"] * max(1, p.size(-2) / p.size(-1))**0.5)

### DistMuon

In [ ]:
# 今回は使わない

class DistMuon(torch.optim.Optimizer):
    """
    Muon最適化関数の分散処理対応バージョン

    ZeRO-2と同様の手法で、最適化関数の状態と勾配をシャーディング（分割）する

    最適化関数の状態は、N分割
    勾配は、各GPUが担当の勾配だけを計算・保持

    Muon: SGD-momentum + (optional) Nesterov, then orthogonalize the 2D update via Newton–Schulz,
    finally apply aspect-ratio scaled step. Performs its own distributed synchronization:
      - reduce_scatter(AVG) for gradient averaging
      - all_gather to replicate updated weights

    Notes:
      * Designed for 2D parameters (e.g., linear/conv kernels reshaped to 2D). Do not use for 0D/1D
        params like embeddings or scalars.
      * Momentum buffers are maintained only on the 'owner' rank for each parameter (rank chosen
        by block-cyclic assignment below). If you checkpoint optimizer state on a single rank,
        consolidate states beforehand.

    Args:
        params: iterable of Tensors
        lr: learning rate
        momentum: momentum coefficient in [0,1)
        nesterov: if True, Nesterov-style update (g <- lerp(g, buf, momentum)); else use buf
        ns_steps: number of Newton–Schulz iterations for the orthogonalization
    """
    def __init__(self, params, lr: float = 0.02, momentum: float = 0.95,
                 nesterov: bool = True, ns_steps: int = 5):
        defaults = dict(lr=lr, momentum=momentum, nesterov=nesterov, ns_steps=ns_steps)
        params = list(params)
        assert all(p.ndim == 2 for p in params), "Muon expects 2D parameters only"
        rank = dist.get_rank()
        # Group all parameters by their shape
        shapes = sorted({p.shape for p in params}) # sort to ensure consistent / deterministic ordering
        param_groups = []
        for shape in shapes:
            group_params = [p for p in params if p.shape == shape]
            device, dtype = group_params[0].device, group_params[0].dtype
            assert all(p.device == device for p in group_params)
            assert all(p.dtype == dtype for p in group_params)
            if rank == 0:
                print(f"Muon: Grouping {len(group_params)} params of shape {shape}, device {device}, dtype {dtype}")
            param_groups.append(dict(params=group_params, zero_buffer=torch.zeros_like(group_params[0])))
        super().__init__(param_groups, defaults)

    @torch.no_grad()
    def step(self):
        rank = dist.get_rank()
        world_size = dist.get_world_size()

        # Ensure all grads exist
        assert all(p.grad is not None for group in self.param_groups for p in group["params"]), "All params must have grads"

        # Kick off all the reduce scatter operations to average up the gradients across all ranks
        all_reduce_futures = []
        for group in self.param_groups:
            params = group["params"]
            zero_buffer = group["zero_buffer"]
            # Go through params in groups of world_size.
            for base_i in range(0, len(params), world_size):
                # The compute owner of each param is rank i % world_size
                owner_idx = base_i + rank
                # each rank stacks up its chunk of world_size params into a list
                rs_input = [p.grad for p in params[base_i:base_i + world_size]]
                # pad rs_input with the zero buffer to complete the group
                rs_input.extend([zero_buffer] * (world_size - len(rs_input)))
                # the output buffer gets strided across the group based on the rank
                rs_output = params[owner_idx].grad if owner_idx < len(params) else torch.empty_like(zero_buffer)
                # reduce scatter the gradients within this group of world_size params
                work = dist.reduce_scatter(rs_output, rs_input, op=dist.ReduceOp.AVG, async_op=True).get_future()
                all_reduce_futures.append(work)

        # Now each rank computes the update and gathers
        future_idx = 0
        all_gather_futures = []
        for group in self.param_groups:
            params = group["params"]
            zero_buffer = group["zero_buffer"]
            # Go through params in groups of world_size.
            for base_i in range(0, len(params), world_size):
                # The compute owner of each param is rank i % world_size
                owner_idx = base_i + rank # calculate the index of the param that this rank owns
                # Wait for the reduce scatter to complete
                all_reduce_futures[future_idx].wait() # possibly later we could use wait_any polling instead
                future_idx += 1
                # Owner computes the Muon update, result is in its param
                if owner_idx < len(params):
                    p = params[owner_idx]
                    g = p.grad  # now averaged across ranks
                    state = self.state[p]
                    if "momentum_buffer" not in state:
                        state["momentum_buffer"] = torch.zeros_like(g)
                    buf: Tensor = state["momentum_buffer"]
                    buf.lerp_(g, 1.0 - group["momentum"])
                    g = g.lerp_(buf, group["momentum"]) if group["nesterov"] else buf
                    g = zeropower_via_newtonschulz5(g, steps=group["ns_steps"])
                    scale = (max(1.0, p.size(-2) / p.size(-1)) ** 0.5)
                    p.add_(g, alpha=-group["lr"] * scale)
                # Replicate updated parameters to all ranks
                ag_input = params[owner_idx] if owner_idx < len(params) else zero_buffer
                ag_output = params[base_i:base_i + world_size]
                ag_output.extend([torch.empty_like(zero_buffer) for _ in range(world_size - len(ag_output))]) # pad
                work = dist.all_gather(ag_output, ag_input, async_op=True).get_future()
                all_gather_futures.append(work)

        # Wait for all work to finish
        torch.futures.collect_all(all_gather_futures).wait()


### DistAdamW

In [ ]:
# 今回は使わない

class DistAdamW(torch.optim.Optimizer):
    """
    AdamW最適化関数の分散処理バージョン

    In the style of ZeRO-2, i.e. sharded optimizer states and gradient reduction
    """
    def __init__(self, param_groups, lr: float = 1e-3, betas: tuple[float, float] = (0.9, 0.999), eps: float = 1e-8, weight_decay: float = 0.01):
        defaults = dict(lr=lr, betas=betas, eps=eps, weight_decay=weight_decay)
        super().__init__(param_groups, defaults)

    @torch.compile
    @torch.no_grad()
    def step(self):
        rank = dist.get_rank()
        world_size = dist.get_world_size()
        reduce_scatter_futures: list[torch.Future] = []
        all_reduce_futures: list[torch.Future] = []
        grad_slices = []
        for group in self.param_groups:
            params: list[Tensor] = group["params"]
            for base_i in range(len(params)):
                grad = params[base_i].grad
                rank_size = grad.shape[0] // world_size
                grad_slice = torch.empty_like(grad[:rank_size])
                reduce_scatter_futures.append(dist.reduce_scatter_tensor(grad_slice, grad, op=dist.ReduceOp.AVG, async_op=True).get_future())
                grad_slices.append(grad_slice)

        idx = 0
        for group in self.param_groups:
            beta1, beta2 = group['betas']
            eps = group['eps']
            wd = group['weight_decay']
            params = group['params']
            for base in range(len(params)):
                reduce_scatter_futures[idx].wait()
                p = params[base]
                rank_size = p.shape[0] // world_size
                p_slice = p[rank * rank_size:(rank + 1) * rank_size]
                lr = group['lr'] * getattr(p, "lr_mul", 1.0)
                state = self.state[p]
                g_slice = grad_slices[idx]
                # State init
                if not state:
                    state['step'] = torch.tensor(0, dtype=torch.int64, device=p.device)
                    state['exp_avg'] = torch.zeros_like(p_slice)
                    state['exp_avg_sq'] = torch.zeros_like(p_slice)
                exp_avg = state['exp_avg']
                exp_avg_sq = state['exp_avg_sq']
                state['step'] += 1
                t = state['step']
                # weight decay
                if wd != 0:
                    eff_weight_decay = lr * wd * getattr(p, "wd_mul", 1.0)
                    p_slice.mul_(1 - eff_weight_decay)
                # update running averages
                exp_avg.mul_(beta1).add_(g_slice, alpha=1 - beta1)
                exp_avg_sq.mul_(beta2).addcmul_(g_slice, g_slice, value=1 - beta2)
                # bias corrections
                bias1 = 1 - beta1 ** t
                bias2 = 1 - beta2 ** t
                # compute step
                denom = exp_avg_sq.sqrt().add_(eps)
                step_size = lr * (torch.sqrt(bias2) / bias1)
                update = exp_avg.div(denom).mul_(step_size)
                p_slice.add_(other=update, alpha=-1.0)
                idx += 1
                all_reduce_futures.append(dist.all_gather_into_tensor(p, p_slice, async_op=True).get_future())
        torch.futures.collect_all(all_reduce_futures).wait()

## モデル実装

In [ ]:
import math
from functools import partial
from dataclasses import dataclass
import torch
import torch.nn as nn
import torch.nn.functional as F

### ヘルパー関数

分散データ並列（Distributed Data Parallel, DDP）で使用する関数

In [ ]:
def is_ddp():
    """
    分散データ並列（Distributed Data Parallel, DDP）が有効かどうか

    Returns:
        bool: DDPが有効な場合True、そうでない場合False
    """

    # RANK環境変数はtorchrunなどによって設定されるプロセスの識別子
    return int(os.environ.get('RANK', -1)) != -1

is_ddp()

In [ ]:
def get_dist_info():
    """
    分散データ並列（Distributed Data Parallel, DDP）訓練の環境変数を取得
    """
    if is_ddp():
        assert all(var in os.environ for var in ['RANK', 'LOCAL_RANK', 'WORLD_SIZE'])

        # グローバルなプロセス識別子
        ddp_rank = int(os.environ['RANK'])

        # ローカル（同一ノード内）のプロセス識別子
        ddp_local_rank = int(os.environ['LOCAL_RANK'])

        # ワールドサイズ（全プロセス数）
        ddp_world_size = int(os.environ['WORLD_SIZE'])
        return True, ddp_rank, ddp_local_rank, ddp_world_size
    else:
        return False, 0, 0, 1

get_dist_info()

シンプルなレイヤー正規化層

In [ ]:
def norm(x):
    """
    平方根平均二乗ノルム正規化（Root Mean Square Normalization, RMSNorm）を適用する

    Args:
        x (Tensor): 入力テンソル
    Returns:
        Tensor: 正規化されたテンソル
    """
    logger.debug(f"RMSNormを適用開始 {x.shape=}")

    # ゲインやバイアスを持たないため高速
    # (4, 2048, 1280) -> (4, 2048, 1280)
    res = F.rms_norm(x, (x.size(-1),))

    logger.debug(f"RMSNorm適用完了 {res.shape=}")
    return res

TransformerのクエリとキーにRoPEを適用する関数

In [ ]:
def apply_rotary_emb(x, cos, sin):
    """
    回転位置埋め込み（RoPE, Rotary Positional Embedding）を適用する

    Args:
        x (Tensor): 入力テンソル
        cos (Tensor): コサイン成分、形状は(1, seq_len, 1, head_dim/2)
        sin (Tensor): サイン成分、形状は(1, seq_len, 1, head_dim/2)
    Returns:
        Tensor: RoPEが適用されたテンソル、形状は(batch_size, seq_len, num_heads, head_dim)
    """
    logger.debug(f"RoPEを適用 {x.shape=}, {x.dtype=} {cos.shape=}, {cos.dtype=} {sin.shape=} {sin.dtype=}")

    assert x.ndim == 4

    # head_dimを2で分割
    d = x.shape[3] // 2

    # 最後の次元を2つに分割
    x1, x2 = x[..., :d], x[..., d:]

    # 2次元回転行列を適用
    y1 = x1 * cos + x2 * sin
    y2 = x1 * (-sin) + x2 * cos
    out = torch.cat([y1, y2], 3) # 最後の次元で結合

    # sinとcosはfloat32で計算されることが多いため、元のデータ型に戻す
    out = out.to(x.dtype)

    logger.debug(f"RoPE適用完了 {out.shape=}")
    return out

### GPTConfig

GPTモデルのデフォルト設定

後で上書きされる

In [ ]:
@dataclass
class GPTConfig:
    sequence_len: int = 1024 # 最大シーケンス長
    vocab_size: int = 50304 # 語彙サイズ
    n_layer: int = 12 # Transformerのレイヤー数
    n_head: int = 6 # アテンションヘッド数
    n_kv_head: int = 6 # キーバリューヘッド数
    n_embd: int = 768 # Transformerの埋め込み次元数

### CausalSelfAttention

Transformerの前半にあるセルフアテンション

In [ ]:
class CausalSelfAttention(nn.Module):
    """
    GQA（Group Query Attention）を実装した因果セルフアテンションモジュール
    """

    def __init__(self, config, layer_idx):
        logger.debug(f"CausalSelfAttentionを初期化開始 {config.n_head=} {config.n_kv_head=} {config.n_embd=} {layer_idx=}")

        super().__init__()

        self.layer_idx = layer_idx

        # 10
        self.n_head = config.n_head

        # 10
        self.n_kv_head = config.n_kv_head

        # 1280
        self.n_embd = config.n_embd

        # 1280 / 10 = 128
        self.head_dim = self.n_embd // self.n_head
        logger.debug(f"{self.head_dim=}")

        assert self.n_embd % self.n_head == 0
        assert self.n_kv_head <= self.n_head and self.n_head % self.n_kv_head == 0

        # 1280 -> 1280
        self.c_q = nn.Linear(self.n_embd, self.n_head * self.head_dim, bias=False)

        # 1280 -> 1280
        self.c_k = nn.Linear(self.n_embd, self.n_kv_head * self.head_dim, bias=False)

        # 1280 -> 1280
        self.c_v = nn.Linear(self.n_embd, self.n_kv_head * self.head_dim, bias=False)

        # 1280 -> 1280
        self.c_proj = nn.Linear(self.n_embd, self.n_embd, bias=False)

        logger.debug("CausalSelfAttentionの初期化完了")

    def forward(self, x, cos_sin, kv_cache):
        logger.debug(f"CausalSelfAttentionの順伝搬を実行 {x.shape=} {x.dtype=} {cos_sin[0].shape=} {cos_sin[0].dtype=} {kv_cache if kv_cache is not None else None=}")

        # (4, 2048, 1280)
        B, T, C = x.size()

        # 1) クエリ・キー・バリューを計算

        # (4, 2048, 1280) -> (4, 2048, 1280) -> (4, 2048, 10, 128)
        q = self.c_q(x).view(B, T, self.n_head, self.head_dim)

        # (4, 2048, 1280) -> (4, 2048, 1280) -> (4, 2048, 10, 128)
        k = self.c_k(x).view(B, T, self.n_kv_head, self.head_dim)

        # (4, 2048, 1280) -> (4, 2048, 1280) -> (4, 2048, 10, 128)
        v = self.c_v(x).view(B, T, self.n_kv_head, self.head_dim)

        # 2) クエリとキーにRoPEを適用

        # (1, 2048, 1, 64), (1, 2048, 1, 64)
        cos, sin = cos_sin

        # (4, 2048, 10, 128), (4, 2048, 10, 128)
        q, k = apply_rotary_emb(q, cos, sin), apply_rotary_emb(k, cos, sin)

        # 3) RMSNormを適用

        # (4, 2048, 10, 128) -> (4, 2048, 10, 128)
        # (4, 2048, 10, 128) -> (4, 2048, 10, 128)
        q, k = norm(q), norm(k)

        # 4) KVキャッシュを適用

        # (4, 2048, 10, 128) -> (4, 10, 2048, 128)
        # (4, 2048, 10, 128) -> (4, 10, 2048, 128)
        # (4, 2048, 10, 128) -> (4, 10, 2048, 128)
        q, k, v = q.transpose(1, 2), k.transpose(1, 2), v.transpose(1, 2)

        # KVキャッシュがある場合（推論時）
        if kv_cache is not None:
            # KVキャッシュを更新し、これまでの全てのKVを取得
            k, v = kv_cache.insert_kv(self.layer_idx, k, v)
            logger.debug(f"KVキャッシュを更新 {k.shape=} {v.shape=}")

        # 5) スケールド・ドットプロダクト・アテンションを計算

        # クエリの数
        # 2048
        Tq = q.size(2)
        logger.debug(f"{Tq=}")

        # キー/バリューの数（キャッシュ内も含む）
        # 2048
        Tk = k.size(2)
        logger.debug(f"{Tk=}")

        # GQAを有効にするかどうか
        # False
        enable_gqa = self.n_head != self.n_kv_head
        logger.debug(f"GQAを有効化: {enable_gqa=}")

        # 訓練時またはクエリ数とキー/バリュー数が等しい場合
        if kv_cache is None or Tq == Tk:
            logger.debug("通常の因果アテンションを適用")
            y = F.scaled_dot_product_attention(q, k, v, is_causal=True, enable_gqa=enable_gqa)

        # 推論時でクエリが1つだけの場合
        elif Tq == 1:
            logger.debug("マスクを使用しない因果アテンションを適用")
            y = F.scaled_dot_product_attention(q, k, v, is_causal=False, enable_gqa=enable_gqa)

        # 推論時で複数のトークンを一度に処理する場合（prefill）
        else:
            logger.debug("マスクを手動で作成して因果アテンションを適用")

            # アテンションマスクを初期化
            attn_mask = torch.zeros((Tq, Tk), dtype=torch.bool, device=q.device)
            logger.debug(f"{attn_mask.shape=}")

            # キャッシュ済みの長さを取得
            prefix_len = Tk - Tq
            logger.debug(f"{prefix_len=}")

            if prefix_len > 0:
                # マスクの左側（キャッシュ部分）をマスクしない
                attn_mask[:, :prefix_len] = True

            # マスクの右側に下三角行列を設定
            attn_mask[:, prefix_len:] = torch.tril(
                torch.ones((Tq, Tq), dtype=torch.bool, device=q.device)
            )

            y = F.scaled_dot_product_attention(q, k, v, attn_mask=attn_mask, enable_gqa=enable_gqa)

        # 6) 出力処理

        # (4, 10, 2048, 128) -> (4, 2048, 10, 128) -> (4, 2048, 1280)
        y = y.transpose(1, 2).contiguous().view(B, T, -1)
        logger.debug(f"アテンション出力を整形 {y.shape=}")

        # (4, 2048, 1280) -> (4, 2048, 1280)
        y = self.c_proj(y)
        logger.debug(f"最終線形変換を適用 {y.shape=}")

        logger.debug(f"CausalSelfAttentionの順伝搬完了 {y.shape=}")
        return y

### MLP


Transformerの後半にあるフィードフォワードネットワーク

In [ ]:
class MLP(nn.Module):
    """
    フィードフォワードネットワーク（FFN）モジュール
    """

    def __init__(self, config):
        logger.debug(f"MLPを初期化 {config.n_embd=}")
        super().__init__()

        # 1280 -> 5120
        self.c_fc = nn.Linear(config.n_embd, 4 * config.n_embd, bias=False)

        # 5120 -> 1280
        self.c_proj = nn.Linear(4 * config.n_embd, config.n_embd, bias=False)

        logger.debug("MLPの初期化完了")

    def forward(self, x):
        logger.debug(f"MLPの順伝搬を実行 {x.shape=} {x.dtype=}")

        x = self.c_fc(x)
        logger.debug(f"{x.shape=}")

        # Squared ReLU活性化関数を適用
        # GELUやSiLU（Swish）よりも高速でメモリ効率が良い
        x = F.relu(x).square()

        x = self.c_proj(x)
        logger.debug(f"MLPの順伝搬完了 {x.shape=}")
        return x

### Block

セルフアテンションとフィードフォワードネットワークを組み合わせたTransformer

In [ ]:
class Block(nn.Module):
    """
    Transformerブロックモジュール
    """

    def __init__(self, config, layer_idx):
        logger.debug(f"Transformer Blockを初期化 {config.n_embd=} {layer_idx=}")
        super().__init__()
        self.attn = CausalSelfAttention(config, layer_idx)
        self.mlp = MLP(config)
        logger.debug("Transformer Blockの初期化完了")

    def forward(self, x, cos_sin, kv_cache):
        logger.debug(f"Transformer Blockの順伝搬を実行 {x.shape=} {x.dtype=} {cos_sin[0].shape=} {cos_sin[0].dtype=} {kv_cache if kv_cache is not None else None=}")

        x = x + self.attn(norm(x), cos_sin, kv_cache)

        x = x + self.mlp(norm(x))

        logger.debug(f"Transformer Blockの順伝搬完了 {x.shape=}")
        return x

### GPT

20層のTransformerを積み重ね、入力にエンコーダー、出力にヘッドをつけたGPTモデル

In [ ]:
class GPT(nn.Module):
    """
    GPTモデル全体
    """

    def __init__(self, config):
        logger.debug(f"GPTモデルを初期化 {config.n_layer=} {config.n_head=} {config.n_embd=} {config.vocab_size=} {config.sequence_len=}")

        super().__init__()

        self.config = config

        # Transformerのエンコーダー部分
        self.transformer = nn.ModuleDict({
            # 単語埋め込み層
            # 65536 -> 1280
            "wte": nn.Embedding(config.vocab_size, config.n_embd),

            # 20層のTransformerブロックを作成
            "h": nn.ModuleList(
                [Block(config, layer_idx) for layer_idx in range(config.n_layer)]
            ),
        })

        # 出力の線形層（Language Model Head）
        # 1280 -> 65536
        self.lm_head = nn.Linear(config.n_embd, config.vocab_size, bias=False)

        # RoPEのシーケンス長を設定
        # KVキャッシュでシーケンスの最大長を超える可能性があるため余裕をもたせる
        # 2048 * 10 = 20480
        self.rotary_seq_len = config.sequence_len * 10
        logger.debug(f"{self.rotary_seq_len=}")

        # ヘッド次元数を計算
        # 1280 / 10 = 128
        head_dim = config.n_embd // config.n_head
        logger.debug(f"{head_dim=}")

        # RoPEのcosとsinを事前計算
        cos, sin = self._precompute_rotary_embeddings(self.rotary_seq_len, head_dim)

        # cosをバッファに登録（state_dictには保存しない）
        self.register_buffer("cos", cos, persistent=False)

        # sinをバッファに登録（state_dictには保存しない）
        self.register_buffer("sin", sin, persistent=False)

        logger.debug("GPTモデルの初期化完了")

    def init_weights(self):
        """
        モデルの重みを初期化する
        """
        logger.debug("GPTモデルの重みを初期化開始")

        # 全てのモジュールに対して初期化関数を適用
        self.apply(self._init_weights)

        # 出力層の重みをゼロに初期化
        torch.nn.init.zeros_(self.lm_head.weight)

        # 全てのTransformerブロックの出力層の重みをゼロに初期化
        for block in self.transformer.h:
            torch.nn.init.zeros_(block.mlp.c_proj.weight)
            torch.nn.init.zeros_(block.attn.c_proj.weight)

        # RoPEのcosとsinを計算
        head_dim = self.config.n_embd // self.config.n_head
        cos, sin = self._precompute_rotary_embeddings(self.rotary_seq_len, head_dim)
        self.cos, self.sin = cos, sin

        # 埋め込み層の重みをbfloat16にダウンキャストし、メモリ使用量を削減
        if self.transformer.wte.weight.device.type == "cuda":
            self.transformer.wte.to(dtype=torch.bfloat16)

        logger.debug("GPTモデルの重みの初期化完了")

    def _init_weights(self, module):
        """
        init_weightsから呼び出される重み初期化関数

        Args:
            module (nn.Module): 初期化するモジュール
        """
        logger.debug(f"モジュールの重みを初期化開始 {module.__class__.__name__=}")

        # 全結合層の場合
        if isinstance(module, nn.Linear):
            # 論文に基づく方法で重みを初期化
            # https://arxiv.org/pdf/2310.17813
            fan_out = module.weight.size(0)
            fan_in = module.weight.size(1)
            std = 1.0 / math.sqrt(fan_in) * min(1.0, math.sqrt(fan_out / fan_in))
            torch.nn.init.normal_(module.weight, mean=0.0, std=std)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)

        # 埋め込み層の場合
        elif isinstance(module, nn.Embedding):
            # 標準正規分布で重みを初期化
            torch.nn.init.normal_(module.weight, mean=0.0, std=1.0)

        logger.debug("モジュールの重みの初期化完了")

    def _precompute_rotary_embeddings(self, seq_len, head_dim, base=10000, device=None):
        """
        RoPEのcosとsinを事前計算する

        Args:
            seq_len (int): シーケンス長
            head_dim (int): ヘッド次元数
            base (int, optional): 逆周波数の基底値。デフォルトは10000。
            device (torch.device, optional): 計算に使用するデバイス。デフォルトはNoneで、自動検出される。
        Returns:
            Tuple[Tensor, Tensor]: 事前計算されたcosとsinのテンソル
        """

        logger.debug(f"RoPEの事前計算を実行 {seq_len=} {head_dim=} {base=}")

        # デバイスが指定されていない場合
        if device is None:
            # 自動検出
            device = self.transformer.wte.weight.device

        # 要素数を半分にするための偶数インデックスを作成
        channel_range = torch.arange(0, head_dim, 2, dtype=torch.float32, device=device)

        # 逆周波数（invert frequency）を計算
        # 角速度と実質的に同じ
        # \theta = base^{-\frac{2i}{head\_dim}}
        # (64,)
        inv_freq = 1.0 / (base ** (channel_range / head_dim))
        logger.debug(f"{inv_freq.shape=}")

        # 文字位置のインデックスを作成
        # (20480,)
        t = torch.arange(seq_len, dtype=torch.float32, device=device)
        logger.debug(f"{t.shape=}")
    
        # 回転角度を計算
        # 回転角度 = 文字位置 * 逆周波数
        # (20480, 64)
        freqs = torch.outer(t, inv_freq)
        logger.debug(f"{freqs.shape=}")

        # 回転角度からcosとsinを計算
        # (20480, 64), (20480, 64)
        cos, sin = freqs.cos(), freqs.sin()

        # bfloat16に変換
        cos, sin = cos.bfloat16(), sin.bfloat16()

        # ブロードキャストのために次元を追加
        # (1, 20480, 1, 64), (1, 20480, 1, 64)
        cos, sin = cos[None, :, None, :], sin[None, :, None, :]

        logger.debug(f"RoPEの事前計算完了 {cos.shape=} {sin.shape=}")
        return cos, sin

    def get_device(self):
        return self.transformer.wte.weight.device

    def estimate_flops(self):
        """
        モデルのトークンあたりのFLOPsを推定する
        FLOPSは1トークン生成するために必要な浮動小数点演算回数
        Chinchillaのスケール則で総訓練ステップ数を導くために必要
        https://arxiv.org/abs/2204.02311
        """
        logger.debug("モデルのFLOPsを推定開始")
    
        # モデルのパラメータ数を計算
        # 560,988,160 = 5.6億
        nparams = sum(p.numel() for p in self.parameters())
        logger.debug(f"{nparams=:,}")

        # 埋め込み層のパラメータ数を取得
        # 83,886,080 = 8,400万
        nparams_embedding = self.transformer.wte.weight.numel()
        logger.debug(f"{nparams_embedding=:,}")

        # レイヤー数l=20
        # ヘッド数h=10
        # クエリの次元q=128
        # シーケンス長t=2048
        l, h, q, t = self.config.n_layer, self.config.n_head, self.config.n_embd // self.config.n_head, self.config.sequence_len
        logger.debug(f"{l=}, {h=}, {q=}, {t=}")

        # FLOPsを計算
        # 3,491,758,080 = 3.5GFLOPs
        num_flops_per_token = 6 * (nparams - nparams_embedding) + 12 * l * h * q * t

        logger.debug(f"モデルのFLOPsの推定完了 {num_flops_per_token=:,}")
        return num_flops_per_token

    def setup_optimizers(self, unembedding_lr=0.004, embedding_lr=0.2, matrix_lr=0.02, weight_decay=0.0):
        """
        最適化関数を設定する

        Args:
            unembedding_lr (float): 出力層の学習率
            embedding_lr (float): 埋め込み層の学習率
            matrix_lr (float): トランスフォーマーの行列パラメータの学習率
            weight_decay (float): 重み減衰（L2正則化）係数
        Returns:
            List[torch.optim.Optimizer]: 設定された最適化関数のリスト
        """
        logger.debug(f"最適化関数を設定開始 {unembedding_lr=} {embedding_lr=} {matrix_lr=} {weight_decay=}")

        # 1) モデルの次元数を取得

        model_dim = self.config.n_embd

        ddp, rank, local_rank, world_size = get_dist_info()

        # トランスフォーマーの行列パラメータ
        matrix_params = list(self.transformer.h.parameters())
        logger.debug(f"{len(matrix_params)=}")

        # トランスフォーマーの埋め込みパラメータ
        embedding_params = list(self.transformer.wte.parameters())

        # 出力層のパラメータ
        lm_head_params = list(self.lm_head.parameters())

        assert len(list(self.parameters())) == len(matrix_params) + len(embedding_params) + len(lm_head_params)

        # 2) 入力の埋め込み層と出力の線形層に対してAdamWを初期化

        # モデルの次元に基づいて学習率をスケーリング
        dmodel_lr_scale = (model_dim / 768) ** -0.5
        if rank == 0:
            logger.debug(f"Scaling the LR for the AdamW parameters ∝1/√({model_dim}/768) = {dmodel_lr_scale:.6f}")

        adam_groups = [
            dict(params=lm_head_params, lr=unembedding_lr * dmodel_lr_scale),
            dict(params=embedding_params, lr=embedding_lr * dmodel_lr_scale),
        ]

        adamw_kwargs = dict(betas=(0.8, 0.95), eps=1e-10, weight_decay=weight_decay)

        AdamWFactory = DistAdamW if ddp else partial(torch.optim.AdamW, fused=True)

        adamw_optimizer = AdamWFactory(adam_groups, **adamw_kwargs)

        # 3) Transformerの行列パラメータに対してMuonを初期化

        muon_kwargs = dict(lr=matrix_lr, momentum=0.95)

        MuonFactory = DistMuon if ddp else Muon

        muon_optimizer = MuonFactory(matrix_params, **muon_kwargs)


        # 4) 2つの最適化関数を1つのリストにまとめる

        optimizers = [adamw_optimizer, muon_optimizer]

        for opt in optimizers:
            for group in opt.param_groups:
                group["initial_lr"] = group["lr"]

        logger.debug("最適化関数の設定完了")
        return optimizers

    def forward(self, idx, targets=None, kv_cache=None, loss_reduction='mean'):
        """
        GPTモデルの順伝搬

        Args:
            idx (Tensor): トークンIDのテンソル、形状は(batch_size, seq_len)
            targets (Tensor, optional): 目標トークンIDのテンソル、形状は(batch_size, seq_len)。デフォルトはNone。
            kv_cache (KVCache, optional): KVキャッシュオブジェクト。デフォルトはNone。
            loss_reduction (str, optional): 損失の集約方法。'mean'または'sum'。デフォルトは'mean'。
        Returns:
            Tensor: 訓練モードの場合は損失テンソル、推論モードの場合はロジットテンソル
            形状は(batch_size, seq_len, vocab_size)
        """

        logger.debug(f"GPTモデルの順伝搬を実行 {idx.shape=} {idx.dtype=} {targets.shape if targets is not None else None=} {kv_cache if kv_cache is not None else None=}")

        B, T = idx.size()

        # 1) RoPEの準備

        assert T <= self.cos.size(1), f"Sequence length grew beyond the rotary embeddings cache: {T} > {self.cos.size(1)}"

        assert idx.device == self.cos.device, f"Rotary embeddings and idx are on different devices: {idx.device} != {self.cos.device}"

        assert self.cos.dtype == torch.bfloat16, "Rotary embeddings must be in bfloat16"

        # KVキャッシュが存在する場合、現在のキャッシュ内の位置にRoPEをオフセットする必要がある
        T0 = 0 if kv_cache is None else kv_cache.get_pos()

        # 事前計算されたRoPEを現在のシーケンス長に切り詰める
        # (1, 2048, 1, 64), (1, 2048, 1, 64)
        cos_sin = self.cos[:, T0:T0+T], self.sin[:, T0:T0+T]
        logger.debug(f"RoPEの切り出し完了 {cos_sin[0].shape=} {cos_sin[1].shape=}")

        # 2) 20層のTransformerの順伝播

        # トークンIDを埋め込みに変換
        x = self.transformer.wte(idx)

        # 最初のRMSNormを適用（Pre-LN）
        x = norm(x)

        # 各Transformerブロックを順番に適用
        for block in self.transformer.h:
            x = block(x, cos_sin, kv_cache)

        # 3) 出力の線形層を適用

        x = norm(x)

        # Logit Softcapping
        # ロジットの値が+/-15を超えないように制限する
        softcap = 15

        # 訓練モードの場合
        if targets is not None:

            # 出力の線形層を適用
            logits = self.lm_head(x)

            # logitsにソフトキャップを適用
            logits = softcap * torch.tanh(logits / softcap)

            # logitsをfloat32にアップキャスト
            logits = logits.float()

            # クロスエントロピー損失を計算
            loss = F.cross_entropy(
                logits.view(-1, logits.size(-1)),
                targets.view(-1),
                ignore_index=-1,
                reduction=loss_reduction
            )

            logger.debug(f"GPTモデルの順伝搬完了（訓練モード） {loss.shape=}")
            return loss

        # 推論モードの場合
        else:
            # 出力の線形層を適用
            logits = self.lm_head(x)

            # logitsにソフトキャップを適用
            logits = softcap * torch.tanh(logits / softcap)

            logger.debug(f"GPTモデルの順伝搬完了（推論モード） {logits.shape=}")
            return logits

    @torch.inference_mode() # 勾配計算を無効化
    def generate(self, tokens, max_tokens, temperature=1.0, top_k=None, seed=42):
        """
        テキストを生成するジェネレーター関数

        Args:
            tokens (List[int]): 初期トークンのリスト
            max_tokens (int): 生成する最大トークン数
            temperature (float, optional): 温度パラメータ。デフォルトは1.0。
            top_k (int, optional): Top-KサンプリングのK値。デフォルトはNoneで無効。
            seed (int, optional): 乱数シード。デフォルトは42。
        Yields:
            int: 生成された各トークンID
        """
        logger.debug(f"テキスト生成を開始 {tokens=} {max_tokens=} {temperature=} {top_k=} {seed=}")

        # 1) 初期化

        assert isinstance(tokens, list)

        device = self.get_device()
        logger.debug(f"{device=}")

        rng = None

        # 温度が設定されている場合、乱数生成器を初期化
        if temperature > 0:
            rng = torch.Generator(device=device)
            rng.manual_seed(seed)

        # バッチ次元を追加
        ids = torch.tensor([tokens], dtype=torch.long, device=device)

        # 2) トークン生成ループ

        # 最大トークン数まで生成
        for _ in range(max_tokens):

            # 1トークンを生成
            # (B, T, vocab_size)
            logits = self.forward(ids)

            # 直近のトークンのロジットを取得
            # (B, vocab_size)
            logits = logits[:, -1, :]

            # Top-Kサンプリングが有効な場合
            if top_k is not None:
                # 上位K個以外のロジットを無限小に設定
                v, _ = torch.topk(logits, min(top_k, logits.size(-1)))
                logits[logits < v[:, [-1]]] = -float('Inf')

            # 温度が設定されている場合
            if temperature > 0:
                # 確率分布をなめらかにしランダムに1つ選択
                logits = logits / temperature
                probs = F.softmax(logits, dim=-1)
                next_ids = torch.multinomial(probs, num_samples=1, generator=rng)

            # 温度が0の場合
            else:
                # 最も高い確率のトークンを選択（Greedy Decoding）
                next_ids = torch.argmax(logits, dim=-1, keepdim=True)

            # 生成したトークンをシーケンスの末尾に追加
            ids = torch.cat((ids, next_ids), dim=1)

            # 生成したトークンをCPUに移動してPythonのintに変換
            token = next_ids.item()

            logger.debug(f"生成トークン: {token}")
            yield token

## データローダー

In [ ]:
from collections import deque
import torch

In [ ]:
def tokenizing_distributed_data_loader(B, T, split, tokenizer_threads=4, tokenizer_batch_size=128, device="cuda"):
    """
    Parquetファイルからテキストをストリーミングし、トークン化し、訓練バッチを作成するデータローダー

    Args:
        B (int): バッチサイズ
        T (int): シーケンス長
        split (str): データセットの分割 ("train" または "val")
        tokenizer_threads (int): トークナイザーのスレッド数
        tokenizer_batch_size (int): トークナイザーのバッチサイズ
        device (str): デバイス ("cuda" または "cpu")

    Yields:
        Tuple[Tensor, Tensor]: 入力テンソルとターゲットテンソルのタプル
    """
    logger.debug(f"分散データローダーを初期化 {B=} {T=} {split=} {tokenizer_threads=} {tokenizer_batch_size=} {device=}")

    # 1) 初期化

    assert split in ["train", "val"], "split must be 'train' or 'val'"

    ddp, ddp_rank, ddp_local_rank, ddp_world_size = get_dist_info()

    # 必要なトークン数を計算
    # 最後の正解トークンを含めるため1を加算
    # 8193
    needed_tokens = B * T + 1
    logger.debug(f"{needed_tokens=}")

    # トークナイザーを取得
    tokenizer = get_tokenizer()

    # BOSトークンを取得
    bos_token = tokenizer.get_bos_token_id()

    # 一度のイテレーションで保持するトークン用のスクラッチバッファ
    # dequeを使用するのは、popleftがO(1)で高速であるため
    token_buffer = deque()

    # 2) ドキュメントのバッチを無限に生成するジェネレーターを作成

    def document_batches():
        """
        生のテキストを無限に供給するためのジェネレーター

        Returns:
            Iterator[List[str]]: テキストドキュメントのバッチを無限に生成するイテレーター
        """
        while True:
            # Parquetファイルからドキュメントのバッチを取得（1024行）
            # DDPが有効な場合、各プロセスが異なるデータを取得（Rank0は0,8,16..., Rank1は1,9,17,...）
            for batch in parquets_iter_batched(split=split, start=ddp_rank, step=ddp_world_size):
                # ミニバッチ（128行）に分割して返す
                for i in range(0, len(batch), tokenizer_batch_size):
                    yield batch[i:i+tokenizer_batch_size]

    batches = document_batches()

    # 3) 文字列を読み込み、トークン化し、整形してバッチを生成

    batch_index = 0
    while True:
        # バッファに十分なトークン（8193）を蓄積するまで繰り返す
        while len(token_buffer) < needed_tokens:
            # 生のテキストドキュメントのバッチを取得（128行）
            doc_batch = next(batches)

            # BOSトークンを先頭に追加してトークン化
            token_lists = tokenizer.encode(doc_batch, prepend=bos_token, num_threads=tokenizer_threads)

            # トークンをバッファの末尾に追加
            for tokens in token_lists:
                token_buffer.extend(tokens)

            batch_index += 1

        # 必要なトークン数（8193）をバッファの先頭から取り出す
        tokens = [token_buffer.popleft() for _ in range(needed_tokens)]

        # pin_memoryを有効化しGPUがメモリに直接アクセスできるように最適化した1次元テンソルを作成
        scratch = torch.tensor(tokens, dtype=torch.int64, pin_memory=(device == "cuda"))

        # 入力シーケンスを作成
        inputs_cpu = scratch[:-1].to(dtype=torch.int32)

        # 正解シーケンスを作成
        # 正解は入力を1トークン右にシフトしたもの
        targets_cpu = scratch[1:]

        # 2Dに変換し（B, T）、非同期（non_blocking=True）でGPUに転送
        inputs = inputs_cpu.view(B, T).to(device=device, dtype=torch.int32, non_blocking=True)
        targets = targets_cpu.view(B, T).to(device=device, dtype=torch.int64, non_blocking=True)
        yield inputs, targets

# 検証
data_loader = tokenizing_distributed_data_loader(
    B=4,
    T=2048,
    split="train",
    device="cuda",
)
first_batch = next(data_loader)
logger.debug(f"データローダーの最初のバッチを取得 {first_batch[0].shape=} {first_batch[1].shape=}")

## 推論エンジン

In [ ]:
import torch
import torch.nn.functional as F
import signal
import warnings
from contextlib import contextmanager
from collections import deque

### KVキャッシュ

アテンションの計算で、一度生成したキーとバリューを再利用し計算を高速化するためのキャッシュ

In [ ]:
class KVCache:
    """
    推論を高速化するためのキー・バリューキャッシュ（KVキャッシュ）モジュール
    """

    def __init__(self, batch_size, num_heads, seq_len, head_dim, num_layers):
        logger.debug(f"KVCacheを初期化開始 {batch_size=} {num_heads=} {seq_len=} {head_dim=} {num_layers=}")

        # モデルの全層にKVキャッシュがある
        self.kv_shape = (num_layers, 2, batch_size, num_heads, seq_len, head_dim)

        self.kv_cache = None

        # キャッシュ内の書き込み位置
        self.pos = 0

        logger.debug("KVCacheの初期化完了")

    def reset(self):
        self.pos = 0

    def get_pos(self):
        return self.pos

    def prefill(self, other):
        """
        KVキャッシュを別のKVキャッシュで事前設定（prefill）する
        オプションでバッチ次元に沿って拡張することも可能
        複数のサンプルを並列に生成する場合に使用

        Args:
            other (KVCache): 事前設定に使用する別のKVキャッシュ
        """
        logger.debug(f"KVキャッシュを事前設定開始 {other.kv_cache is not None=}")

        # 1) 形状を検証

        assert self.kv_chache is None, "KVキャッシュが空でない場合、事前設定できません"
        assert other.kv_cache is not None, "事前設定に使用するKVキャッシュが空です"

        # 各次元を検証
        for ix, (dim1, dim2) in enumerate(zip(self.kv_shape, other.kv_shape)):
            # Transformerの層数、K/V、ヘッド数、ヘッド次元の検証
            if ix in [0, 1, 3, 5]:
                assert dim1 == dim2, f"Batch dim mismatch: {dim1} != {dim2}"

            # バッチサイズの検証
            elif ix == 2:
                # バッチサイズは拡張可能
                assert dim1 == dim2 or dim2 == 1, f"Batch dim mismatch: {dim1} != {dim2}"

            # シーケンス長の検証
            elif ix == 4:
                # シーケンス長は長い必要がある
                assert dim1 >= dim2, f"Seq len mismatch: {dim1} < {dim2}"

        # 2) キャッシュを初期化

        dtype, device = other.kv_cache.dtype, other.kv_cache.device
        self.kv_cache = torch.empty(self.kv_shape, dtype=dtype, device=device)

        # 3) データをコピー

        # バッチ次元に沿って複製
        self.kv_cache[:, :, :, :, :other.pos, :] = other.kv_cache

        # 4) 書き込み位置を更新

        self.pos = other.pos

        logger.debug("KVキャッシュの事前設定完了")

    def insert_kv(self, layer_idx, k, v):
        """
        KVキャッシュに新しいキーとバリューを挿入し、これまでの全てのキーとバリューを返す

        Args:
            layer_idx (int): 現在のTransformer層のインデックス
            k (Tensor): 新しいキーのテンソル、形状は (B, H, T_add, D)
            v (Tensor): 新しいバリューのテンソル、形状は (B, H, T_add, D)
        Returns:
            Tuple[Tensor, Tensor]: これまでの全てのキーとバリューのビュー、形状は (B, H, T_total, D)
        """
        logger.debug(f"KVキャッシュに挿入を実行 {layer_idx=} {k.shape=} {v.shape=} {self.pos=}")

        # 1) 初期化

        # KVキャッシュが初期化されていない場合
        if self.kv_cache is None:
            # KVキャッシュを初期化
            # データ型とデバイスが必要なため遅延初期化（lazy initialization）
            self.kv_cache = torch.empty(self.kv_shape, dtype=k.dtype, device=k.device)

        # Insert new keys/values to the cache and return the full cache so far

        # 現在の位置と追加するトークン数を取得
        B, H, T_add, D = k.size()

        # 書き込み開始位置t0と終了位置t1を計算 
        t0, t1 = self.pos, self.pos + T_add
        logger.debug(f"{t0=} {t1=}")

        # 書き込みの終了位置が初期化時のシーケンス長を超える場合、キャッシュを拡張
        if t1 > self.kv_cache.size(4):
            # 1024トークン分拡張
            t_needed = t1 + 1024

            # 1024の倍数に切り上げ
            t_needed = (t_needed + 1023) & ~1023

            additional_shape = list(self.kv_cache.shape)

            additional_shape[4] = t_needed - self.kv_cache.size(4)
            additional_cache = torch.empty(additional_shape, dtype=k.dtype, device=k.device)
            self.kv_cache = torch.cat([self.kv_cache, additional_cache], dim=4).contiguous()
            self.kv_shape = self.kv_cache.shape

        # 2) キーとバリューをキャッシュに挿入

        # キーをキャッシュに挿入
        self.kv_cache[layer_idx, 0, :, :, t0:t1] = k

        # バリューをキャッシュに挿入
        self.kv_cache[layer_idx, 1, :, :, t0:t1] = v

        # 3) これまでの全てのキーとバリューを返す

        # これまでの全てのキーを取得
        key_view = self.kv_cache[layer_idx, 0, :, :, :t1]

        # これまでの全てのバリューを取得
        value_view = self.kv_cache[layer_idx, 1, :, :, :t1]

        # 最後のTransformer層の場合、書き込み位置を更新
        if layer_idx == self.kv_cache.size(0) - 1:
            self.pos = t1

        logger.debug(f"KVキャッシュへの挿入完了 {key_view.shape=} {value_view.shape=} {self.pos=}")
        return key_view, value_view

エンジンのgenerateメソッドで使用するサンプル関数

In [ ]:
@torch.inference_mode() # 勾配計算を無効化
def sample_next_token(logits, rng, temperature=1.0, top_k=None):
    """
    次の1つのトークンをサンプリングする
    バッチ処理、温度パラメータ、Top-Kサンプリングに対応

    Args:
        logits (Tensor): ロジットのテンソル、形状は (B, vocab_size)
        rng (Generator): 乱数生成器
        temperature (float): 温度パラメータ
        top_k (int, optional): Top-KサンプリングのK値
    Returns:
        Tensor: サンプリングされた次のトークン、形状は (B, 1)
    """
    logger.debug(f"次のトークンをサンプリング開始 {logits.shape=} {temperature=} {top_k=}")

    assert temperature >= 0.0, "温度パラメータは0以上である必要があります"

    # 温度が0の場合
    if temperature == 0.0:
        # 最も高い確率のトークンを選択（Greedy Decoding）
        res = torch.argmax(logits, dim=-1, keepdim=True)

    # Top-Kサンプリングが有効な場合
    if top_k is not None:
        k = min(top_k, logits.size(-1))

        # 上位K個のロジットとインデックスを取得
        vals, idx = torch.topk(logits, k, dim=-1)

        # 温度を適用しロジットを調整
        vals = vals / temperature

        # ロジットを確率に変換
        probs = F.softmax(vals, dim=-1)

        # 上位K個の中からランダムに1つ選択
        choice = torch.multinomial(probs, num_samples=1, generator=rng)

        # 元の語彙インデックスにマッピング
        res = idx.gather(1, choice)

    # 標準的なサンプリングの場合
    else:
        # 温度を適用しロジットを調整
        logits = logits / temperature

        # ロジットを確率に変換
        probs = F.softmax(logits, dim=-1)

        # 確率分布からランダムに1つ選択
        res = torch.multinomial(probs, num_samples=1, generator=rng)

    logger.debug(f"次のトークンのサンプリング完了 {res.shape=}")
    return res

ツール呼び出し中など生成中のサンプル（Row）のフラグを管理するクラス

In [ ]:
class RowState:
    """
    生成中の各行（サンプル）の状態を追跡するクラス
    """
    def __init__(self, current_tokens=None):

        # これまでに生成されたトークンIDの完全なリスト
        self.current_tokens = current_tokens or []

        # サンプリングしたトークンを無視し強制的に挿入するトークンのキュー
        # <|output_start|>などの特殊トークンの挿入に使用
        self.forced_tokens = deque()

        # <|python_start|>トークンを生成した時にTrueになるフラグ
        self.in_python_block = False

        # self.in_python_blockがTrueの間に生成されたトークンIDのリスト
        # Pythonのコードに対応するトークンID
        self.python_expr_tokens = []

        # 生成が完了したかどうかのフラグ
        # <|assistant_end|>や<|bos|>が生成された場合にTrueになる
        self.completed = False

生成中のツール呼び出しを安全に行うためのヘルパー関数

In [ ]:
@contextmanager
def timeout(duration, formula):
    """
    指定した時間を超えた場合に例外を発生させるコンテキストマネージャー
    Pythonのコードを実行する際のタイムアウトに使用

    Args:
        duration (int): タイムアウトまでの時間（秒）
        formula (str): 実行するPythonコードの文字列
    Yields:
        None
    """

    def timeout_handler(signum, frame):
        raise Exception(f"'{formula}': timed out after {duration} seconds")

    # OSのアラームシグナルとハンドラーを関連付ける
    signal.signal(signal.SIGALRM, timeout_handler)

    # タイマーを開始（duration秒後にシグナルを送信）
    signal.alarm(duration)

    yield

    # タイマーを停止
    signal.alarm(0)

In [ ]:
def eval_with_timeout(formula, max_time=3):
    """
    Pythonのコードを安全に実行する

    Args:
        formula (str): 実行するPythonコードの文字列
        max_time (int): タイムアウト時間（秒）
    Returns:
        Any: 実行結果、失敗した場合はNone
    """
    try:
        # 3秒以内に実行を完了できない場合は例外を発生させる
        with timeout(max_time, formula):

            # SyntaxWarningを無視
            with warnings.catch_warnings():
                warnings.simplefilter("ignore", SyntaxWarning)

                # コードを実行
                return eval(formula)

    except Exception as e:
        signal.alarm(0)
        return None

# 検証
eval_with_timeout("44 * 99")

生成中のツール呼び出しでPythonのコードを安全に実行する関数

In [ ]:
def use_calculator(expr):
    """
    Pythonのコードを検証して安全に実行する
    推論時のツール呼び出しで使用（Tool Use）

    Args:
        expr (str): コード
    Returns:
        Any: 評価結果、危険な式の場合はNone
    """
    logger.debug(f"コードを評価開始 {expr=}")

    # 1) コードが数式の場合

    # 数値からカンマを削除
    expr = expr.replace(",", "")

    # 純粋な数学式かどうかをチェック
    if all([x in "0123456789*+-/.() " for x in expr]):

        # 指数演算子が含まれている場合は拒否
        if "**" in expr:
            return None

        # 安全に評価
        return eval_with_timeout(expr)

    # 2) コードが文字列操作の場合

    # [や{などの危険な文字を含む場合は拒否
    allowed_chars = "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789'\"()._ "
    if not all([x in allowed_chars for x in expr]):
        return None

    # 危険な文字列を含む場合は拒否 
    dangerous_patterns = ['__', 'import', 'exec', 'eval', 'compile', 'open', 'file',
                         'input', 'raw_input', 'globals', 'locals', 'vars', 'dir',
                         'getattr', 'setattr', 'delattr', 'hasattr']
    expr_lower = expr.lower()
    if any(pattern in expr_lower for pattern in dangerous_patterns):
        return None

    # countメソッド以外は拒否
    if '.count(' not in expr:
        return None

    # 安全に評価
    return eval_with_timeout(expr)

生成中のフラグを管理しなら推論を行うエンジン

In [ ]:
class Engine:
    """
    モデルの推論を実行するインターフェイス
    """

    def __init__(self, model, tokenizer):
        logger.debug(f"Engineを初期化開始 {model=} {tokenizer=}")
        
        self.model = model

        # ツール呼び出しで使用
        self.tokenizer = tokenizer

        logger.debug("Engineの初期化完了")

    @torch.inference_mode() # 勾配計算を無効化
    def generate(self, tokens, num_samples=1, max_tokens=None, temperature=1.0, top_k=None, seed=42):
        """
        トークンを1つずつストリーミングで生成するジェネレータ

        Args:
            tokens (List[int]): プロンプトのトークンIDのリスト
            num_samples (int): 生成するサンプル数（行数）
            max_tokens (int, optional): 生成する最大トークン数
            temperature (float): 温度パラメータ
            top_k (int, optional): Top-KサンプリングのK値
            seed (int): 乱数シード
        Yields:
            Tuple[List[int], List[int]]: 各行の次のトークンIDのリストと対応するマスクのリスト
        """
        logger.debug(f"ストリーミング生成を開始 {tokens=} {num_samples=} {max_tokens=} {temperature=} {top_k=} {seed=}")

        # 1) 初期化

        # 入力トークンの型を検証
        assert isinstance(tokens, list) and isinstance(tokens[0], int), "expecting list of ints"

        # デバイスを取得
        device = self.model.get_device()

        # 乱数生成器を初期化
        rng = torch.Generator(device=device)
        rng.manual_seed(seed)

        # 特殊トークンのIDを取得
        get_special = lambda s: self.tokenizer.encode_special(s)
        python_start = get_special("<|python_start|>")
        python_end = get_special("<|python_end|>")
        output_start = get_special("<|output_start|>")
        output_end = get_special("<|output_end|>")
        assistant_end = get_special("<|assistant_end|>") # if sampled, ends row
        bos = self.tokenizer.get_bos_token_id() # if sampled, ends row

        # 2) プロンプトのトークンを処理

        # KVキャッシュを初期化
        m = self.model.config
        kv_model_kwargs = {"num_heads": m.n_kv_head, "head_dim": m.n_embd // m.n_head, "num_layers": m.n_layer}
        kv_cache_prefill = KVCache(
            batch_size=1, # バッチサイズ1
            seq_len=len(tokens), # プロンプトの長さ
            **kv_model_kwargs, # KVキャッシュ設定
        )

        # トークンをテンソルに変換
        ids = torch.tensor([tokens], dtype=torch.long, device=device)
        logger.debug(f"{ids.shape=}")

        # モデルを順伝播してKVキャッシュを事前設定（prefill）
        logits = self.model.forward(ids, kv_cache=kv_cache_prefill)
        logger.debug(f"{logits.shape=}")

        # 最後のトークン（次の単語の予測）を抽出
        logits = logits[:, -1, :]
        logger.debug(f"{logits.shape=}")

        # 最初の1トークンをサンプリング    
        # (B, 1)
        next_ids = sample_next_token(logits, rng, temperature, top_k)
        logger.debug(f"{next_ids.shape=}")

        # サンプリングされたトークンをPythonのリストに変換
        sampled_tokens = next_ids[:, 0].tolist()
        logger.debug(f"{sampled_tokens=}")

        # 2) KVキャッシュの複製

        # 生成用のKVキャッシュを初期化
        kv_length_hint = (len(tokens) + max_tokens) if max_tokens is not None else self.model.config.sequence_len
        kv_cache_decode = KVCache(
            batch_size=num_samples,
            seq_len=kv_length_hint,
            **kv_model_kwargs,
        )
        logger.debug(f"{num_samples=} {kv_length_hint=}")

        # プロンプトで事前設定されたKVキャッシュを複製
        kv_cache_decode.prefill(kv_cache_prefill)

        # Prefill用のKVキャッシュは不要になったので削除
        del kv_cache_prefill

        # 3) 各サンプルの状態を初期化

        row_states = [RowState(tokens.copy()) for _ in range(num_samples)]
        logger.debug(f"{len(row_states)=}")

        # 4) 生成ループ

        # max_tokensに達するか、全サンプルが完了するまでトークンを生成
        num_generated = 0
        first_iteration = True
        while True:

            # 最大シーケンス長に達した場合、停止
            if max_tokens is not None and num_generated >= max_tokens:
                break

            # 全サンプルが完了した場合、停止
            if all(state.completed for state in row_states):
                break

            # 最初の生成ステップの場合
            if first_iteration:

                # プリフィルでサンプリングしたトークンを使用
                # すべてのサンプルにブロードキャスト
                sampled_tokens = [sampled_tokens[0]] * num_samples
                logger.debug(f"最初の生成ステップ {sampled_tokens=}")

                first_iteration = False

            # 2回目以降の生成ステップの場合
            else:

                # 順伝播を行い、各サンプルの次のトークンのロジットを計算
                # (B, T, vocab_size)
                logits = self.model.forward(ids, kv_cache=kv_cache_decode)
                logger.debug(f"2回目移行の生成ステップ {logits.shape=}")

                # 直近に生成されたトークンのロジットを抽出
                # (B, vocab_size)
                logits = logits[:, -1, :]
                logger.debug(f"{logits.shape=}")

                # 各サンプルの次のトークンをサンプリング
                # (B, 1)
                next_ids = sample_next_token(logits, rng, temperature, top_k)
                logger.debug(f"{next_ids.shape=}")

                # リストに変換
                sampled_tokens = next_ids[:, 0].tolist()
                logger.debug(f"{sampled_tokens=}")

            # それぞれのサンプルを処理
            token_column = [] # contains the next token id along each row
            token_masks = [] # contains the mask (was it sampled (1) or forced (0)?) along each row
            for i, state in enumerate(row_states):

                # 強制的に挿入するトークンがあるかを確認
                is_forced = len(state.forced_tokens) > 0

                # マスクを作成 
                # 0なら強制トークン、1ならサンプリングトークン
                token_masks.append(0 if is_forced else 1)

                # 強制トークンを優先して次のトークンを決定
                next_token = state.forced_tokens.popleft() if is_forced else sampled_tokens[i]

                # トークン列に次のトークンを追加
                token_column.append(next_token)

                # 状態の現在のトークン列に次のトークンを追加
                state.current_tokens.append(next_token)

                # 次のトークンが<|assistant_end|>または<|bos|>の場合
                if next_token == assistant_end or next_token == bos:
                    # 生成完了フラグを立てる
                    state.completed = True

                # <|python_start|>トークンが生成された場合
                if next_token == python_start:
                    # Pythonコードブロックに入るフラグを立てる
                    state.in_python_block = True
                    # Pythonコードトークンのリストを初期化
                    state.python_expr_tokens = []

                # <|python_end|>トークンが生成された場合
                elif next_token == python_end and state.in_python_block:
                    # Pythonコードブロックから出るフラグを下ろす
                    state.in_python_block = False

                    # ツール呼び出しを実行
                    if state.python_expr_tokens:
                        # トークン列をPythonコードの文字列にデコード
                        expr = self.tokenizer.decode(state.python_expr_tokens)

                        # ツールを呼び出す
                        result = use_calculator(expr)

                        # 結果がある場合
                        if result is not None:
                            # 結果をトークン化
                            result_tokens = self.tokenizer.encode(str(result))

                            # 特殊トークンで囲んで強制挿入キューに追加
                            # <|output_start|> result <|output_end|>
                            state.forced_tokens.append(output_start)
                            state.forced_tokens.extend(result_tokens)
                            state.forced_tokens.append(output_end)

                    # Pythonコードトークンのリストをクリア
                    state.python_expr_tokens = []

                # Pythonコードブロック内の場合
                elif state.in_python_block:
                    # 生成されたトークンをPythonコードトークンのリストに追加
                    state.python_expr_tokens.append(next_token)

            # 各行の次のトークン列とマスクをストリーミング出力
            yield token_column, token_masks

            # 生成トークン数を加算
            num_generated += 1

            # 次のループのためにトークンIDをテンソルに変換
            ids = torch.tensor(token_column, dtype=torch.long, device=device).unsqueeze(1)

    def generate_batch(self, tokens, num_samples=1, **kwargs):
        """
        非ストリーミング版

        Args:
            tokens (List[int]): プロンプトのトークンIDのリスト
            num_samples (int): 生成するサンプル数（行数）
            **kwargs: generateメソッドに渡す追加の引数
        Returns:
            Tuple[List[List[int]], List[List[int]]]: 各サンプルの生成されたトークン列と対応するマスクのリスト
        """
        logger.debug(f"バッチ生成を開始 {tokens=} {num_samples=} {kwargs=}")

        # 1) 初期化

        # ストップトークンのIDを取得
        assistant_end = self.tokenizer.encode_special("<|assistant_end|>")
        bos = self.tokenizer.get_bos_token_id()

        # 入力プロンプトを各サンプルにコピー
        results = [tokens.copy() for _ in range(num_samples)]

        # 各サンプルのマスクを初期化
        # 0は強制的に挿入されたトークン、1はサンプリングされたトークン
        masks = [[0] * len(tokens) for _ in range(num_samples)]

        # 各サンプルの生成完了フラグを初期化
        completed = [False] * num_samples

        # 2) 生成ループ

        # ストリーミングでトークンを生成
        for token_column, token_masks in self.generate(tokens, num_samples, **kwargs):

            # 各サンプルの生成されたトークンとマスクを収集
            for i, (token, mask) in enumerate(zip(token_column, token_masks)):

                # 未完了の場合
                if not completed[i]:

                    # 終了トークンの場合
                    if token == assistant_end or token == bos:

                        # 完了フラグを立てる
                        completed[i] = True

                    # 通常のトークンの場合
                    else:
                        # トークンを追加
                        results[i].append(token)

                        # マスクを追加
                        masks[i].append(mask)

            # すべての完了フラグが立っている場合、ループを終了
            if all(completed):
                break

        logger.debug(f"バッチ生成完了 {results=} {masks=}")
        return results, masks

## 事前学習

### ハイパーパラメータ

ハイパーパラメータの決定例:

1. 予算を決める
    - 8台のA100を40時間
1. 予算内で訓練できそうなモデルサイズを経験的に決める
    - 32層（depth=32）
1. VRAMの限界を探る
    - device_batch_sizeを変えて、メモリ不足（OOM, Out of Memory）にならない最大バッチサイズを探す
    - nvidia-smiでMFU（Model Flops Utilization, GPUメモリ使用率）が50%以下だと良好
1. 総訓練時間の検証
    - モデルサイズからパラメータ数を計算
    - Chinchillaのスケーリング則で訓練に必要な総トークン数を求める（38B）
        - 訓練に必要な総トークン数 = 20 * パラメータ数
    - 総トークン数から総ステップ数を自動計算（71,680ステップ）
    - 1ステップあたりの時間を実測（1.572秒）
    - 訓練に必要な時間を計算（71,680 * 1.572 = 31.3時間）
    - 残りの10時間をファインチューニング用に割り当てる
1. 必要データ量の計算
    - トークナイザーの性能を確認（-4.8文字/トークン）
    - 必要文字数を計算（訓練に必要な総トークン数 * 4.8 = 185B文字
    - 1シャードあたりの文字数を設定 250M文字
    - 必要シャード数 185B / 250M = 740シャード
    - 安全のため 800シャードに切り上げ（不足すると読み直すため過学習のリスク）

#### 基本設定

In [ ]:
# 訓練に使用するGPU数（プロセス数）
# デフォルトは8
NPROC_PER_NODE=1

# Weights & Biasesの実行名
# "dummy"の場合はロギングを無効にする
run = "dummy"

# 実行デバイス
# ""の場合はCUDA > MPS > CPUの順で自動検出する
device_type = "cuda"

#### モデル設定

In [ ]:
# Transformerのレイヤー数
# デフォルトは20（d20）
depth = 20

# 最大シーケンス長
# デフォルトは2048
max_seq_len = 2048

# 訓練ステップ数
# デフォルトは-1（無効）
num_iterations = -1

# 目標とする総計算量（FLOPs）
# デフォルトは-1.0（無効）
target_flops = -1.0

# モデルのパラメータ数に対する訓練トークン数の比率
# num_iterationsよりも優先され、設定されている場合は訓練ステップ数が自動計算される
# Chinchillaのスケーリング則（訓練トークン数 = 20 * パラメータ数）に基づく
# デフォルトは20で、-1の場合は無効
target_param_data_ratio = 20

#### 最適化設定

In [ ]:
# 1GPUあたりのバッチサイズ
# メモリ不足が発生した場合は減らす
# デフォルトは32
# 4 -> VRAM 31GB使用
# 1 -> VRAM 10GB程度
device_batch_size = 1 if gpu_name == "T4" else 4

# 目標とする総バッチサイズ（トークン数単位）
# 勾配蓄積（Gradient Accumulation）のステップ数を自動計算するために使用
# デフォルトは524,288トークン = 32 * 2048 * 8 = device_batch_size * max_seq_len * world_size
total_batch_size = 524_288

# 入力の埋め込み層のAdamWの学習率
# デフォルトは0.2
embedding_lr = 0.2

# 最終層（lm_head）のAdamWの学習率
# デフォルトは0.004
unembedding_lr = 0.004

# AdamWの重み減衰係数（L2正則化）
# デフォルトは0.0
weight_decay = 0.0

# Transformerの行列パラメータのMuonの学習率
# デフォルトは0.02
matrix_lr = 0.02

# 勾配クリッピング
# 0.0の場合は無効
# デフォルトは1.0
grad_clip = 1.0

# 学習率のウォームアップの割合
# デフォルトは0.0
warmup_ratio = 0.0

# 学習率のウォームダウンの割合
# デフォルトは0.2
warmdown_ratio = 0.2

# 最終的な学習率を初期学習率の何倍にするか
# デフォルトは0.0
final_lr_frac = 0.0

#### 評価

訓練中の評価は2つ:

- BPB評価（Bits per bytes, トークンではなくバイト数の平均で損失を計算する手法）
- COREメトリックベンチマーク（22個の大規模言語モデル用のベンチマーク）

In [ ]:
# BPB評価のインターバルステップ数
# デフォルトは250
eval_every = 10 if OTAMESHI_MODE else 250

# 検証データの評価に使うトークン数
# デフォルトは20 * 524,288トークン（10Mトークン）
eval_tokens = 20 * 524288

# コアメトリック評価のインターバルステップ数
# -1の場合は無効
# デフォルトは2000
core_metric_every = 10 if OTAMESHI_MODE else 2000

# コアメトリック評価をする際の各タスクあたりの最大サンプル数
# デフォルトは500
core_metric_max_per_task = 500

# モデルからサンプルを生成するインターバルステップ数
# デフォルトは2000
sample_every = 10 if OTAMESHI_MODE else 2000

#### 出力

In [ ]:
# チェックポイントを保存するディレクトリ名
# ""の場合はdepthに基づいて自動生成
model_tag = ""

# このファイルで定義されたグローバル変数をリストアップし、辞書にまとめる
# WandBのログに記録される
config_keys = [
    k for k,v in globals().items() if not k.startswith('_') and isinstance(v, (int, float, bool, str))
]
user_config = {k: globals()[k] for k in config_keys} # will be useful for logging
user_config.keys()

#### 自動計算

ハイパーパラメーターらから必要なパラメータを自動計算する

In [ ]:
def autodetect_device_type():
    """
    デバイスを自動検出する
    """
    if torch.cuda.is_available():
        device_type = "cuda"
    elif torch.backends.mps.is_available():
        device_type = "mps"
    else:
        device_type = "cpu"

    logger.debug(f"自動検出されたデバイス: {device_type}")
    return device_type

device_type = autodetect_device_type() if device_type == "" else device_type
logger.debug(f"実行デバイス: {device_type}")

In [ ]:
def compute_init(device_type="cuda"): # cuda|cpu|mps
    """
    PyTorchの実行環境を初期化・設定する

    Args:
        device_type (str): 使用するデバイスの種類 ("cuda", "mps", "cpu")
    Returns:
        Tuple[bool, int, int, int, torch.device]:
            - ddp (bool): Distributed Data Parallelが有効かどうか
            - ddp_rank (int): DDPのランク
            - ddp_local_rank (int): DDPのローカルランク
            - ddp_world_size (int): DDPのワールドサイズ
            - device (torch.device): 使用するデバイス
    """

    # 1) デバイスの検証

    assert device_type in ["cuda", "mps", "cpu"], "Invalid device type atm"

    if device_type == "cuda":
        assert torch.cuda.is_available(), "Your PyTorch installation is not configured for CUDA but device_type is 'cuda'"

    if device_type == "mps":
        assert torch.backends.mps.is_available(), "Your PyTorch installation is not configured for MPS but device_type is 'mps'"

    # 2) 再現性の設定

    torch.manual_seed(42)
    if device_type == "cuda":
        torch.cuda.manual_seed(42)

    # 速度が低下する可能性があるため、完全な再現性はスキップ
    # torch.use_deterministic_algorithms(True)

    # 3) 計算精度の設定

    if device_type == "cuda":
        # 行列計算の精度をTF32に設定
        torch.set_float32_matmul_precision("high")

    # 4) 分散環境の設定

    ddp, ddp_rank, ddp_local_rank, ddp_world_size = get_dist_info()

    # DDPが有効な場合
    if ddp and device_type == "cuda":
        # ローカルランクに基づいてデバイスを設定
        device = torch.device("cuda", ddp_local_rank)
        torch.cuda.set_device(device)

        # 分散通信ライブラリを初期化
        dist.init_process_group(backend="nccl", device_id=device)
        dist.barrier()
    # DDPが無効な場合
    else:
        # 単一デバイスを使用
        device = torch.device(device_type)

    if ddp_rank == 0:
        logger.debug(f"ワールドサイズ {ddp_world_size=}")

    return ddp, ddp_rank, ddp_local_rank, ddp_world_size, device

ddp, ddp_rank, ddp_local_rank, ddp_world_size, device = compute_init(device_type)

In [ ]:
# マスタープロセスの判定
# マスタープロセスは、ログ出力・モデルの保存・WandBの記録などを担当
master_process = ddp_rank == 0
logger.debug(f"マスタープロセス {master_process=} {ddp_rank=}")

In [ ]:
# 自動混合精度（Auto Mixed Precision, AMP）のコンテキストマネージャーを設定
autocast_ctx = torch.amp.autocast(device_type=device_type, dtype=torch.bfloat16) \
    if device_type == "cuda" else nullcontext()

logger.debug(f"自動混合精度を有効化 {device_type == 'cuda'}")

In [ ]:
# CUDAの操作を同期する関数を設定

synchronize = torch.cuda.synchronize if device_type == "cuda" else lambda: None
logger.debug(f"CUDAの操作を同期する {device_type == 'cuda'}")

In [ ]:
# 訓練中の最大VRAM使用量を取得する関数を設定

get_max_memory = torch.cuda.max_memory_allocated if device_type == "cuda" else lambda: 0
logger.debug(f"最大VRAM使用量を取得する {device_type == 'cuda'}")

In [ ]:
# Weights & Biasesの初期化

class DummyWandb:
    def __init__(self):
        pass
    def log(self, *args, **kwargs):
        pass
    def finish(self):
        pass

use_dummy_wandb = run == "dummy" or not master_process

# WandBを初期化
wandb_run = DummyWandb() if use_dummy_wandb else wandb.init(project="nanochat", name=run, config=user_config)
logger.debug(f"WandBを使用する {not use_dummy_wandb}")

In [ ]:
# トークナイザーを取得
tokenizer = get_tokenizer()
logger.debug(f"使用するトークナイザー {tokenizer.__class__.__name__}")

vocab_size = tokenizer.get_vocab_size()
logger.debug(f"語彙数 {vocab_size=}")

In [ ]:
def get_token_bytes(device="cpu"):
    """
    BPB評価で使用するトークンとバイト列のマッピングを読み込む

    Args:
        device (str): デバイス ("cpu" または "cuda")
    Returns:
        Tensor: トークンバイト列のテンソル
    """
    import torch
    base_dir = get_base_dir()
    tokenizer_dir = os.path.join(base_dir, "tokenizer")
    token_bytes_path = os.path.join(tokenizer_dir, "token_bytes.pt")
    assert os.path.exists(token_bytes_path), f"Token bytes not found at {token_bytes_path}? It gets written by tok_train.py"
    with open(token_bytes_path, "rb") as f:
        token_bytes = torch.load(f, map_location=device)
    return token_bytes

token_bytes = get_token_bytes(device=device)
token_bytes.shape

In [ ]:
# Transformerの層の数
# デフォルトは20
num_layers = depth

# モデルの埋め込み次元
# 一般的にはアスペクト比は64でモデルサイズが大きくなると128に増やす
# デフォルトは1280
model_dim = depth * 64

# アテンションヘッドの数
# 各ヘッドの次元を128に設定
# デフォルトは10
num_heads = max(1, (model_dim + 127) // 128)

# キーとバリューのヘッド数
# GQA（Group Query Attention）を有効にする場合はnum_headsより小さく設定
# デフォルトは0（無効）
num_kv_heads = num_heads

logger.debug(f"モデル設定 {num_layers=} {model_dim=} {num_heads=} {num_kv_heads=}")

In [ ]:
# 勾配蓄積のステップ数を計算

# 1回の順伝播・逆伝播で1GPUが処理するトークンの総数
# 8192
tokens_per_fwdbwd = device_batch_size * max_seq_len

# 1回の順伝播・逆伝播で全GPUが処理するトークンの総数
# 8192
world_tokens_per_fwdbwd = tokens_per_fwdbwd * ddp_world_size

# 合計バッチサイズ（トークン数単位）が処理する総数で割り切れるかを検証
assert total_batch_size % world_tokens_per_fwdbwd == 0

# 勾配蓄積のステップ数
# 64
grad_accum_steps = total_batch_size // world_tokens_per_fwdbwd

logger.debug(f"最適化設定 {device_batch_size=} {tokens_per_fwdbwd=} {world_tokens_per_fwdbwd=} {total_batch_size=} {grad_accum_steps=}")

### 初期化

#### モデル

In [ ]:
# GPTConfigのキーワード引数を設定
model_config_kwargs = dict(
    sequence_len=max_seq_len,
    vocab_size=vocab_size,
    n_layer=num_layers,
    n_head=num_heads,
    n_kv_head=num_kv_heads,
    n_embd=model_dim
)

# メタデバイス上で実体化せずにモデルを作成
with torch.device("meta"):
    model_config = GPTConfig(**model_config_kwargs)
    model = GPT(model_config)

# メモリ領域を確保
model.to_empty(device=device)

# モデルの重みを初期化
model.init_weights()

# コンパイル前のモデルへの参照を取得
# state_dictを保存するために使用
orig_model = model

# モデルをコンパイル
# dynamic=Falseの場合、入力テンソルの形状が固定される
model = torch.compile(model, dynamic=False)

# モデルの総パラメータ数を計算
# 560,988,160（5.6億パラメータ）
num_params = sum(p.numel() for p in model.parameters())

# 1トークンを処理するために必要な計算量（FLOPs）を推定
# 3,491,758,080（3.5GFLOPs）
num_flops_per_token = model.estimate_flops()

logger.debug(f"モデルの初期化完了 {num_params=:,} {num_flops_per_token=:,}")

In [ ]:
# 総訓練ステップ数の計算

assert num_iterations > 0 or target_param_data_ratio > 0 or target_flops > 0

# ハイパーパラメータの優先度で訓練ステップ数を決定

if num_iterations > 0:
    logger.debug(f"ユーザー指定の訓練ステップ数を使用 {num_iterations=:,}")

elif target_flops > 0:

    # 目標FLOPs / （1トークンあたりのFLOPs * 1ステップあたりのトークン数）
    num_iterations = round(target_flops / (num_flops_per_token * total_batch_size))
    logger.debug(f"目標とする総計算量（FLOPs）から計算された訓練ステップ数 {num_iterations=:,}")

elif target_param_data_ratio > 0:

    # Chinchillaのスケーリング則に基づいて必要なトークン数を計算
    # 11,219,763,200 = 112億トークン
    target_tokens = target_param_data_ratio * num_params
    logger.debug(f"{target_tokens=:,}")

    # 必要なトークン数 / 1ステップあたりのトークン数
    # 11,219,763,200 / 524,288 = 21,400ステップ
    num_iterations = target_tokens // total_batch_size

    logger.debug(f"Chinchillaのスケーリング則から計算された訓練ステップ数 {num_iterations=:,}")
else:
    raise ValueError("No training horizon specified")

# 総訓練トークン数
# 11,219,763,200 = 112億トークン
total_tokens = total_batch_size * num_iterations

logger.debug(f"総訓練トークン数を計算 {total_tokens=:,}")
logger.debug(f"Chinchillaの比率 {total_tokens / num_params:.2f} tokens per param") # 20
logger.debug(f"訓練の推定総計算量 {num_flops_per_token * total_tokens=:e}") # 3.917670e+19 = 39.2PFLOPs

#### 最適化関数

In [ ]:
# 最適化関数を初期化

optimizers = model.setup_optimizers(
    unembedding_lr=unembedding_lr,
    embedding_lr=embedding_lr,
    matrix_lr=matrix_lr,
    weight_decay=weight_decay
)

adamw_optimizer, muon_optimizer = optimizers

#### データローダー

In [ ]:
# データローダーを初期化

base_dir = get_base_dir()
tokens_dir = os.path.join(base_dir, "tokenized_data")

train_loader = tokenizing_distributed_data_loader(
    device_batch_size,
    max_seq_len,
    split="train",
    device=device
)

build_val_loader = lambda: tokenizing_distributed_data_loader(
    device_batch_size,
    max_seq_len,
    split="val",
    device=device
)

# 検証
x, y = next(train_loader)
x.shape, y.shape

#### 学習スケジューラ

In [ ]:
# 学習率スケジューラを定義

def get_lr_multiplier(it):
    """
    学習率の倍率を取得

    Args:
        it (int): 現在の訓練ステップ数
    Returns:
        float: 学習率の倍率
    """

    warmup_iters = round(warmup_ratio * num_iterations)
    warmdown_iters = round(warmdown_ratio * num_iterations)

    # ウォームアップの場合
    if it < warmup_iters:
        return (it + 1) / warmup_iters

    # 定常状態の場合
    elif it <= num_iterations - warmdown_iters:
        return 1.0

    # ウォームダウンの場合
    else:
        progress = (num_iterations - it) / warmdown_iters
        return progress * 1.0 + (1 - progress) * final_lr_frac

#### モーメンタム

In [ ]:
# Muonのモーメンタムを動的に調整する関数

def get_muon_momentum(it):
    """
    Muonのモーメンタムを取得

    Args:
        it (int): 現在の訓練ステップ数
    Returns:
        float: Muonのモーメンタム
    """

    # 0.0から1.0まで300ステップで線形に増加する割合を計算
    frac = min(it / 300, 1)

    # 0.85から0.95まで線形に増加するモーメンタムを計算
    momentum = (1 - frac) * 0.85 + frac * 0.95

    return momentum

### BPB評価

In [ ]:
import math
import torch
import torch.distributed as dist

In [ ]:
@torch.no_grad()
def evaluate_bpb(model, batches, steps, token_bytes):
    """
    BPB (bits per byte)でモデルを評価
    合計損失と合計バイト数を計算し、それらを割り算して正規化する
    """

    # 1) 初期化

    # 損失を記録するテンソルを初期化
    total_nats = torch.tensor(0.0, dtype=torch.float32, device=model.get_device())

    # モデルが予測したトークンの合計バイト数を記録するテンソルを初期化
    total_bytes = torch.tensor(0, dtype=torch.int64, device=model.get_device())

    # 検証データローダーのイテレーターを作成
    batch_iter = iter(batches)

    # 2) 評価ループ

    # 指定されたステップ数だけ評価を実行
    for _ in range(steps):

        # 入力シーケンスと正解シーケンスのバッチを取得
        x, y = next(batch_iter)

        # モデルの順伝搬を実行し、各トークンの損失を計算
        # loss_reduction='none'により、各トークンの損失が返される
        # (B, T)
        loss2d = model(x, y, loss_reduction='none')

        # フラット化
        # (B*T,)
        loss2d = loss2d.view(-1)

        # 正解トークンをフラット化
        # (B*T,)
        y = y.view(-1)

        # 正解シーケンスにignore_index（-1）が含まれている場合、それを除いて集計
        if (y.int() < 0).any():

            # マスクを作成
            valid = y >= 0

            # 無効なインデックスを0に置き換え
            y_safe = torch.where(valid, y, torch.zeros_like(y))

            # マスクされていないトークンをバイト数に変換
            num_bytes2d = torch.where(
                valid,
                token_bytes[y_safe],
                torch.zeros_like(y, dtype=token_bytes.dtype)
            )

            # ignore_indexと特殊トークンを除外して（num_bytes2d > 0）、損失を集計
            total_nats += (loss2d * (num_bytes2d > 0)).sum()

            # バイト数を集計
            total_bytes += num_bytes2d.sum()

        # 正解シーケンスにignore_indexが含まれていない場合、普通に集計
        else:
            # トークンをバイト数に変換
            num_bytes2d = token_bytes[y]

            # 特殊トークンを除外して（num_bytes2d > 0）、損失を集計
            total_nats += (loss2d * (num_bytes2d > 0)).sum()

            # バイト数を集計
            total_bytes += num_bytes2d.sum()


    # DDPが有効な場合、全プロセスで合計を集約
    world_size = dist.get_world_size() if dist.is_initialized() else 1
    if world_size > 1:
        dist.all_reduce(total_nats, op=dist.ReduceOp.SUM)
        dist.all_reduce(total_bytes, op=dist.ReduceOp.SUM)

    # 最終的なBPBを計算して返す
    total_nats = total_nats.item()
    total_bytes = total_bytes.item()
    if total_bytes == 0:
        return float('inf')
    
    # BPB = 総損失 / ln(2) / 総バイト数
    bpb = total_nats / (math.log(2) * total_bytes)
    return bpb

### COREベンチマーク

[DataComp-LM](https://arxiv.org/abs/2406.11794)

In [ ]:
import os
import sys
import time
import json
import random
import yaml
from contextlib import nullcontext

import pandas as pd
import torch
import random

from jinja2 import Template
import torch
import torch.distributed as dist

In [ ]:
# 評価バンドルをダウンロード

EVAL_BUNDLE_URL = "https://karpathy-public.s3.us-west-2.amazonaws.com/eval_bundle.zip"

base_dir = get_base_dir()
eval_bundle_dir = os.path.join(base_dir, "eval_bundle")

if not os.path.exists(eval_bundle_dir):
    # 評価バンドルをダウンロードして解凍
    import requests
    import zipfile
    from io import BytesIO

    logger.debug(f"評価バンドルをダウンロード中 {EVAL_BUNDLE_URL}...")
    response = requests.get(EVAL_BUNDLE_URL)
    with zipfile.ZipFile(BytesIO(response.content)) as z:
        z.extractall(base_dir)
    logger.debug(f"評価バンドルのダウンロードと解凍が完了 {eval_bundle_dir}")
else:
    logger.debug(f"ダウンロード済みの評価バンドル {os.listdir(eval_bundle_dir)}")

In [ ]:
# 設定ファイルを確認

config_path = os.path.join(eval_bundle_dir, "core.yaml")
with open(config_path, 'r') as f:
    config = yaml.safe_load(f)

len(config["icl_tasks"]), config["icl_tasks"]

# 実行する22のタスクのリストと設定が記載されている
# label: タスク名
# dataset_uri: データセットの場所（JSONL)
# num_fewshot: プロンプトに含める回答例の数
# inc_task_type: 評価方法
#   - language_modeling: 言語モデリング（コンテキストから続きの単語を予測するタスク）
#   - multiple_choice: 多肢選択式（コンテキストから最も適切な選択肢を選ぶタスク）
#   - schema: スキーマ（選択肢から最も適切なコンテキストを選ぶタスク）

- World Knowledge（世界の知識）
    - jeopardy: 10ショット, 言語モデリング
    - bigbench_qa_wikidata: 10ショット, 言語モデリング
    - arc_easy: 10ショット, 多肢選択
    - arc_challenge: 10ショット, 多肢選択
- Commonsense Reasoning（常識的推論）
    - copa: 0ショット, 多肢選択
    - commonsense_qa: 10ショット, 多肢選択
    - piqa: 10ショット, 多肢選択
    - openbook_qa: 0ショット, 多肢選択
- Language Understanding（言語理解）
    - hellaswag_zeroshot: 0ショット, 多肢選択
    - lambada_openai: 0ショット, 言語モデリング
    - hellaswag: 10ショット, 多肢選択
    - winograd: 0ショット, スキーマ
    - winogrande: 0ショット, スキーマ
    - bigbench_language_identification: 10ショット, 多肢選択
- Symbolic Problem Solving（記号的問題解決）
    - bigbench_dyck_languages: 10ショット, 言語モデリング
    - agi_eval_lsat_ar: 3ショット, 多肢選択
    - bigbench_cs_algorithms: 10ショット, 言語モデリング
    - bigbench_operators: 10ショット, 言語モデリング
    - bigbench_repeat_copy_logic: 10ショット, 言語モデリング
-  Reading Comprehension（読解）
    - squad: 10ショット, 言語モデリング
    - coqa: 0ショット, 言語モデリング
    - boolq: 10ショット, 多肢選択

In [ ]:
# 言語モデリングタスクのデータセットを確認

data_base_path = os.path.join(eval_bundle_dir, "eval_data")
data_path = os.path.join(data_base_path,  "world_knowledge", "jeopardy_all.jsonl")
with open(data_path, 'r') as f:
    lm_sample = json.loads(f.readline().strip())

lm_sample

# コンテキスト: この海軍司令官は1929年11月28日から29日にかけて、リトルアメリカの基地から南極点までを往復旅行した
# 続き: リチャード・バード提督
    
# モデルの回答と評価の流れ:
# 1. コンテキストから、続きなしのプロンプトと続きありのプロンプトを作成
# 2. 続きなしのプロンプトをモデルに入力し、続きを生成
# 3. 生成された続きを続きありのプロンプトと比較し、完全一致かを判定

In [ ]:
# 多肢選択式タスクのデータセットを確認

data_base_path = os.path.join(eval_bundle_dir, "eval_data")
data_path = os.path.join(data_base_path,  "language_understanding", "hellaswag.jsonl")
with open(data_path, 'r') as f:
    mc_sample = json.loads(f.readline().strip())

mc_sample

# クエリ: 一人の男が屋根の上に座っている。彼は
# 選択肢0: スキー板をラップで包んでいる
# 選択肢1: 水平なタイルを剥がしている
# 選択肢2: ルービックキューブを持っている
# 選択肢3: 屋根材を引き剥がし始めている
# 正解: 選択肢3

# モデルの回答と評価の流れ
# 1. クエリと選択肢を結合した4つのプロンプトを作成
# 2. 選択肢の部分の平均損失を計算
# 3. 最も低い損失を示す選択肢をモデルの選択とする
# 4. モデルの選択と正解を比較し、正誤を判定

In [ ]:
# スキーマタスクのデータセットを確認

data_base_path = os.path.join(eval_bundle_dir, "eval_data")
data_path = os.path.join(data_base_path,  "language_understanding", "winograd_wsc.jsonl")
with open(data_path, 'r') as f:
    schema_sample = json.loads(f.readline().strip())

schema_sample

# 選択肢1: 市議会議員たちはデモ隊に許可を与えなかった。なぜなら市議会議員たちは
# 選択肢2: デモ隊は市議会議員たちに許可を与えなかった。なぜならデモ隊は
# 共通の続き: 暴力を恐れていたからだ。
# 正解: 1

# モデルの回答と評価の流れ:
# 1. 選択肢と共通の続きを結合した2つのプロンプトを作成しトークン化
# 2. 順伝播で共通の続きの部分の平均損失を計算
# 3. 最も低い損失を示す選択肢をモデルの選択とする
# 4. モデルの選択と正解を比較し、正誤を判定

In [ ]:
def render_prompts_mc(item, continuation_delimiter, fewshot_examples=None):
    """
    多肢選択（multiple choice）タスク用のプロンプトをレンダリングする

    このタスクは、選択肢ごとに別々のプロンプトを生成し、最も低い損失を持つ選択肢をモデルの選択と見なす

    Args:
        item (dict): 評価アイテムの辞書
        continuation_delimiter (str): 続きの区切り文字列
        fewshot_examples (List[dict], optional): 少数ショットの例のリスト
    Returns:
        List[str]: 各選択肢に対応する完全なプロンプトのリスト
    """

    # Jinja2テンプレートを定義
    # お手本 + 質問 + 続きの区切り文字列 + 選択肢
    template_str = """
{%- for example in fewshot_examples -%}
{{ example.query }}{{ continuation_delimiter }}{{ example.choices[example.gold] }}

{% endfor -%}
{{ item.query }}{{ continuation_delimiter }}{{ choice }}""".strip()

    # テンプレートをコンパイル
    template = Template(template_str)

    # プロンプトをレンダリング
    fewshot_examples = fewshot_examples or []
    context = {
        'fewshot_examples': fewshot_examples,
        'continuation_delimiter': continuation_delimiter,
        'item': item
    }
    prompts = [template.render(choice=choice, **context) for choice in item['choices']]
    return prompts

# 検証

mc_prompts = render_prompts_mc(
    mc_sample,
    continuation_delimiter="<ANSWER>",
    fewshot_examples=[
        {
            "query": "QUERY",
            "choices": [
                "CHOICE_1",
                "CHOICE_2",
                "CHOICE_3",
                "CHOICE_4"
            ],
            "gold": 0
        },
    ]
)
mc_prompts

In [ ]:
def render_prompts_schema(item, continuation_delimiter, fewshot_examples=None):
    """
    スキーマタスク用のプロンプトをレンダリングする
    スキーマタスクは、与えられたコンテキストに基づいて特定の形式で応答を生成するタスク
    レンダリングは、「異なるコンテキスト + 共通の続き（suffix）」の形式をとる

    Args:
        item (dict): スキーマタスクのデータ項目
        continuation_delimiter (str): 続きの区切り文字列
        fewshot_examples (List[dict], optional): 少数ショットの例のリスト
    Returns:
        List[str]: レンダリングされたプロンプトのリスト
    """

    # スキーマタスク用のテンプレート文字列
    # お手本（fewshot examples） + 異なるコンテキスト + 共通の続き（suffix）
    template_str = """
{%- for example in fewshot_examples -%}
{{ example.context_options[example.gold] }}{{ continuation_delimiter }}{{ example.continuation }}

{% endfor -%}
{{ context }}{{ continuation_delimiter }}{{ item.continuation }}""".strip()

    # テンプレートを作成
    template = Template(template_str)

    # 少数ショットの例を初期化
    fewshot_examples = fewshot_examples or []

    # レンダリング
    context = {
        'fewshot_examples': fewshot_examples,
        'continuation_delimiter': continuation_delimiter,
        'item': item
    }
    prompts = [template.render(context=context_option, **context)
               for context_option in item['context_options']]

    return prompts

# 検証

schema_prompts = render_prompts_schema(
    schema_sample,
    continuation_delimiter="<ANSWER>",
    fewshot_examples=[{
        "context_options": [
            "CONTEXT_1 ",
            "CONTEXT_2 "
        ],
        "continuation": "CONTINUATION",
        "gold": 0
    }]  
)
schema_prompts

In [ ]:
def render_prompts_lm(item, continuation_delimiter, fewshot_examples=None):
    """
    言語モデリングタスク用のプロンプトをレンダリングする

    Args:
        item (dict): 評価アイテム
        continuation_delimiter (str): 続きの区切り文字列
        fewshot_examples (list, optional): 少数ショットの例のリスト
    Returns:
        list: 続きなしと続きありの2つのプロンプト

    注意: テンプレート内のコンテキストを手動でトリミングしています。
          一部のデータセットでは末尾に空白が含まれているようで、
          それを避けるためです。


    Render complete prompt for a language modeling task.
    Notice that we manually trim the context in the template,
    which in some datasets seems to have trailing whitespace (which we don't want).
    """

    # Jinja2形式のテンプレート文字列を作成
    # fewshot example + ... + 問題文（コンテキスト）+ 続きの区切り文字列 + 続き（オプション）
    # 一部のデータセットでは末尾に空白が含まれるためtrimフィルターを使用
    template_str = """
{%- for example in fewshot_examples -%}
{{ example.context | trim }}{{ continuation_delimiter }}{{ example.continuation }}

{% endfor -%}
{{ item.context | trim }}{{ continuation_delimiter }}{% if include_continuation %}{{ item.continuation }}{% endif %}""".strip()

    # テンプレートをコンパイル
    template = Template(template_str)

    fewshot_examples = fewshot_examples or []

    context = {
        'fewshot_examples': fewshot_examples,
        'continuation_delimiter': continuation_delimiter,
        'item': item
    }

    # 正解の続きが含まれないプロンプトをレンダリング
    prompt_without = template.render(include_continuation=False, **context)

    # 正解の続きが含まれるプロンプトをレンダリング
    prompt_with = template.render(include_continuation=True, **context)

    # prompot_withoutの末尾に空白が残るバグがあるため明示的に空白を削除
    prompt_without = prompt_without.strip()

    return [prompt_without, prompt_with]

# 検証

lm_prompts = render_prompts_lm(
    lm_sample,
    continuation_delimiter="<ANSWER>",
    fewshot_examples=[{
        "context": "SAMPLE_CONTEXT",
        "continuation": "SAMPLE_CONTINUATION"
    }
])
lm_prompts

In [ ]:
def find_common_length(token_sequences, direction='left'):
    """
    トークンIDのリストを受け取り、共通の接頭辞（prefix）または接尾辞（suffix）の長さを返す

    多肢選択タスクは質問文が共通（接頭辞） = right
    スキーマタスクは続きの部分が共通（接尾辞） = left

    Args:
        token_sequences (List[List[int]]): トークンIDのリストのリスト
        direction (str): 'left'（接頭辞）または 'right'（接尾辞）
    Returns:
        int: 共通の接頭辞または接尾辞の長さ
    """

    # 最小のシーケンス長を取得
    min_len = min(len(seq) for seq in token_sequences)

    # インデックスの範囲を設定
    # leftの場合 [0, 1, 2, ..., min_len-1]
    # rightの場合 [-1, -2, -3, ..., -min_len]
    indices = {
        'left': range(min_len),
        'right': range(-1, -min_len-1, -1)
    }[direction]

    # 共通の長さを見つける
    for i, idx in enumerate(indices):

        # 基準となるトークンを取得
        token = token_sequences[0][idx]

        # すべてのシーケンスでidx番目のトークンが異なった場合
        if not all(seq[idx] == token for seq in token_sequences):
            # それまでに一致した回数を返す（共通の長さ）
            return i

    # 不一致が見つからなかった場合、最も短いシーケンス長を返す
    return min_len

# 検証
mc_tokens = tokenizer(mc_prompts, prepend=tokenizer.get_bos_token_id())
answer_start_idx = find_common_length(mc_tokens, direction='left')
answer_start_idx
# 4つのプロンプトのうち、32番目のトークンからが選択肢部分（損失計算対象）

In [ ]:
def stack_sequences(tokens, pad_token_id):
    """
    長さが異なるトークンのリストを1つのテンソルにまとめる（stack）

    Args:
        tokens (List[List[int]]): トークンIDのリスト
        pad_token_id (int): パディングに使用するトークンID
    Returns:
        Tensor: パディングされたトークンIDのテンソル
    """

    # バッチサイズと最大シーケンス長を計算
    bsz, seq_len = len(tokens), max(len(x) for x in tokens)

    # パッディングで初期化したテンソルを作成
    # (batch_size, seq_len)
    input_ids = torch.full((bsz, seq_len), pad_token_id, dtype=torch.long)

    # トークンIDをテンソルに上書き
    for i, x in enumerate(tokens):
        input_ids[i, :len(x)] = torch.tensor(x, dtype=torch.long)

    return input_ids

# 検証
pad_token_id = tokenizer.get_bos_token_id()
print(len(mc_tokens[0]), len(mc_tokens[1]), len(mc_tokens[2]), len(mc_tokens[3]))
stack_sequences(mc_tokens, pad_token_id).shape
# 異なる長さのトークン列がパディングされる

In [ ]:
def batch_sequences_mc(tokenizer, prompts):
    """
    多肢選択（multiple choice）タスクのプロンプトをバッチ化する
    プロンプトをトークン化し、損失計算対象の開始・終了インデックスを取得する
    プロンプトに含まれるコンテキストは同じだが、選択肢が異なる形式

    Args:
        tokenizer: トークナイザーのインスタンス
        prompts (List[str]): 各選択肢のプロンプトのリスト
    Returns:
        Tuple[List[List[int]], List[int], List[int]]:
            - tokens (List[List[int]]): 各プロンプトのトークンIDのリスト
            - start_indices (List[int]): 各プロンプトの損失計算対象の開始インデックスのリスト
            - end_indices (List[int]): 各プロンプトの損失計算対象の終了インデックスのリスト
    """
    # プロンプトをトークン化
    tokens = tokenizer(prompts, prepend=tokenizer.get_bos_token_id())

    # 共通のコンテキスト部分の長さを取得し、損失対象の開始インデックスとする
    answer_start_idx = find_common_length(tokens, direction='left')

    # 開始インデックスを複製
    start_indices = [answer_start_idx] * len(prompts)

    # 終了インデックスを各プロンプトの長さとする
    end_indices = [len(x) for x in tokens]

    return tokens, start_indices, end_indices

# 検証
mc_tokens, mc_start_indices, mc_end_indices = batch_sequences_mc(tokenizer, mc_prompts)
len(mc_tokens), mc_start_indices, mc_end_indices
# 4つの選択肢それぞれに対して、損失計算対象の開始・終了インデックスが得られる

In [ ]:
def batch_sequences_schema(tokenizer, prompts):
    """
    スキーマタスクを評価するためにプロンプトをバッチ化する

    Args:
        tokenizer: トークナイザーオブジェクト
        prompts (List[str]): プロンプトのリスト
    Returns:
        Tuple[List[List[int]], List[int], List[int]]:
            - tokens: 各プロンプトのトークンIDのリスト
            - start_indices: 評価対象の続きの開始インデックスのリスト
            - end_indices: 評価対象の続きの終了インデックスのリスト
    """

    # プロンプトを全てトークン化
    tokens = tokenizer(prompts, prepend=tokenizer.get_bos_token_id())

    # コンテキストの開始・終了インデックスを算出
    suffix_length = find_common_length(tokens, direction='right')

    # 各プロンプトの長さを取得
    end_indices = [len(x) for x in tokens]

    # 接尾辞の長さを引いて続きの開始インデックスを算出
    start_indices = [ei - suffix_length for ei in end_indices]

    return tokens, start_indices, end_indices

# 検証
schema_tokens, schema_start_indices, schema_end_indices = batch_sequences_schema(tokenizer, schema_prompts)
len(schema_tokens), schema_start_indices, schema_end_indices
# 2つの選択肢それぞれに対して、損失計算対象の開始・終了インデックスが得られる

In [ ]:
def batch_sequences_lm(tokenizer, prompts):
    """
    言語モデリングタスクを評価するためにプロンプトをバッチ化する

    Args:
        tokenizer: トークナイザー
        prompts (List[str]): 「文脈のみ」と「文脈とその続きの正解」を含む2つの文字列のリスト
    Returns:
        Tuple[List[List[int]], List[int], List[int]]
        トークンIDのリスト、開始インデックスのリスト、終了インデックスのリスト
    """

    # 2つの文字列のリストをトークン化
    tokens = tokenizer(prompts, prepend=tokenizer.get_bos_token_id())

    # 「文脈のみ」と「文脈とその続きの正解」のトークン列を取得
    tokens_without, tokens_with = tokens

    # 損失計算対象の開始・終了インデックスを取得
    start_idx, end_idx = len(tokens_without), len(tokens_with)

    assert start_idx < end_idx, "prompt without is supposed to be a prefix of prompt with"
    assert tokens_without == tokens_with[:start_idx], "prompt without is supposed to be a prefix of prompt with"

    # 正解の続きを含むトークン列、損失計算対象の開始・終了インデックスを返す
    return [tokens_with], [start_idx], [end_idx]

# 検証
lm_tokens, lm_start_indices, lm_end_indices = batch_sequences_lm(tokenizer, lm_prompts)
len(lm_tokens), lm_start_indices, lm_end_indices
# 1つのトークン列に対して、損失計算対象の開始・終了インデックスが得られる

In [ ]:
@torch.no_grad() # 勾配計算を無効化
def forward_model(model, input_ids):
    """
    モデルの順伝播を行い、トークンごとの損失と予測を取得する

    Args:
        model: モデル
        input_ids (Tensor): (B, T)のトークンIDテンソル
    Returns:
        Tuple[Tensor, Tensor]: (B, T)の損失テンソルとargmax予測テンソル

    Take BxT tensor of token ids, return BxT tensor of losses and argmax predictions.
    The last column of losses is set to nan because we don't have autoregressive targets there.
    """

    batch_size, seq_len = input_ids.size()

    # モデルの順伝播を実行して出力を取得
    # (batch_size, seq_len, vocab_size)
    outputs = model(input_ids)

    # 入力シーケンスを左に1つシフトして正解トークンを取得
    target_ids = torch.roll(input_ids, shifts=-1, dims=1)

    # クロスエントロピー損失を計算
    losses = torch.nn.functional.cross_entropy(
        outputs.view(batch_size * seq_len, -1), # (batch_size*seq_len, vocab_size)
        target_ids.view(batch_size * seq_len), # (batch_size*seq_len,)
        reduction='none', # トークンごとの損失を取得
    ).view(batch_size, seq_len) # (batch_size, seq_len)

    # 最後の位置は正解トークンが存在しないため、損失をNaNに設定
    losses[:, -1] = float('nan')

    # 各位置でのargmax予測を取得
    # (batch_size, seq_len)
    predictions = outputs.argmax(dim=-1)

    return losses, predictions

In [ ]:
@torch.no_grad() # 勾配計算を無効化
def evaluate_example(idx, model, tokenizer, data, device, task_meta):
    """
    単一のサンプルを評価し、正解ならTrue、そうでなければFalseを返す

    Args:
        idx (int): 評価するサンプルのインデックス
        model (torch.nn.Module): 評価に使用するモデル
        tokenizer: トークナイザー
        data (list): 評価データのリスト
        device (torch.device): 使用するデバイス
        task_meta (dict): タスクのメタ情報
    Returns:
        bool: 正解ならTrue、そうでなければFalse
    """

    # 1) Few-shotサンプルの準備

    # サンプルを取得
    item = data[idx]

    # タスクタイプを取得
    task_type = task_meta['task_type']

    # Few-shotの数を取得
    num_fewshot = task_meta['num_fewshot']

    # 
    continuation_delimiter = task_meta['continuation_delimiter']

    # 現在のサンプル以外のfew-shotサンプルをランダムに選択
    fewshot_examples = []
    if num_fewshot > 0:
        rng = random.Random(1234 + idx)
        available_indices = [i for i in range(len(data)) if i != idx]
        fewshot_indices = rng.sample(available_indices, num_fewshot)
        fewshot_examples = [data[i] for i in fewshot_indices]

    # 2) プロンプトのレンダリング

    # 多肢選択の場合
    if task_type == 'multiple_choice':

        # 多肢選択のプロンプトをレンダリング
        prompts = render_prompts_mc(item, continuation_delimiter, fewshot_examples)

        # トークン化されたプロンプトと損失計算対象の範囲を取得
        tokens, start_idxs, end_idxs = batch_sequences_mc(tokenizer, prompts)

    # スキーマの場合 
    elif task_type == 'schema':

        # スキーマのプロンプトをレンダリング
        prompts = render_prompts_schema(item, continuation_delimiter, fewshot_examples)

        # トークン化されたプロンプトと損失計算対象の範囲を取得
        tokens, start_idxs, end_idxs = batch_sequences_schema(tokenizer, prompts)

    # 言語モデリングの場合
    elif task_type == 'language_modeling':

        # 言語モデリングのプロンプトをレンダリング
        prompts = render_prompts_lm(item, continuation_delimiter, fewshot_examples)

        # トークン化されたプロンプトと損失計算対象の範囲を取得
        tokens, start_idxs, end_idxs = batch_sequences_lm(tokenizer, prompts)
    else:
        raise ValueError(f"Unsupported task type: {task_type}")

    # 3) トークンのスタック（バッチ化）

    # モデルに最大シーケンス長の属性がある場合
    if hasattr(model, 'max_seq_len') and model.max_seq_len is not None:

        max_tokens = model.max_seq_len
        new_tokens, new_start_idxs, new_end_idxs = [], [], []

        # 各シーケンスを確認し、必要に応じてトークンをトリミング
        for t, s, e in zip(tokens, start_idxs, end_idxs):

            # 最大シーケンス長を超える場合
            if len(t) > max_tokens:

                # 切り詰めるトークン数を計算
                num_to_crop = len(t) - max_tokens

                # 最初のトークンを捨てて切り詰める
                new_tokens.append(t[-max_tokens:])

                # 損失計算対象の範囲のインデックスを調整
                new_start_idxs.append(s - num_to_crop)
                new_end_idxs.append(e - num_to_crop)
                assert s - num_to_crop >= 0, "this should never happen right?"
                assert e - num_to_crop >= 0, "this should never happen right?"

            # 最大シーケンス長を超えない場合
            else:
                # 変更なし
                new_tokens.append(t)
                new_start_idxs.append(s)
                new_end_idxs.append(e)

        # 上書き
        tokens, start_idxs, end_idxs = new_tokens, new_start_idxs, new_end_idxs

    # パディングトークンIDを取得
    pad_token_id = tokenizer.get_bos_token_id()

    # パディングトークンで長さを揃えてバッチにまとめる
    input_ids = stack_sequences(tokens, pad_token_id)

    # デバイスに転送
    input_ids = input_ids.to(device)

    # 4) モデルの順伝播

    # モデルの順伝播を実行し、各トークンの自己回帰的損失とargmax予測を取得
    losses, predictions = forward_model(model, input_ids)

    # 5) 正解の判定

    # 言語モデリングの場合
    if task_type == 'language_modeling':

        # 損失対象範囲のインデックスを取得
        si = start_idxs[0]
        ei = end_idxs[0]

        # 予測したトークンを取得
        predicted_tokens = predictions[0, si-1:ei-1]

        # 正解のトークンを取得
        actual_tokens = input_ids[0, si:ei]

        # 予測が正解と完全に一致するかを判定
        is_correct = torch.all(predicted_tokens == actual_tokens).item()

    # 多肢選択・スキーマの場合
    elif task_type in ['multiple_choice', 'schema']:

        # 各選択肢の平均損失を計算
        mean_losses = [losses[i, si-1:ei-1].mean().item()
                        for i, (si, ei) in enumerate(zip(start_idxs, end_idxs))]

        # 最も損失が低い選択肢のインデックスを取得（モデルが尤もらしいと判断した選択肢）
        pred_idx = mean_losses.index(min(mean_losses))

        # 予測が正解と一致するかを判定
        is_correct = pred_idx == item['gold']
    else:
        raise ValueError(f"Unsupported task type: {task_type}")

    return is_correct

In [ ]:
def evaluate_task(model, tokenizer, data, device, task_meta):
    """
    1つのタスクの全てのデータを処理し、最終的な正解率を計算する
    分散処理にも対応

    Args:
        model: 評価するモデル
        tokenizer: トークナイザー
        data: タスクのデータセット
        device: 使用するデバイス
        task_meta: タスクのメタ情報
    Returns:
        float: タスクの正解率
    """

    # 1) 分散環境の初期化

    # 分散環境のランク（プロセスID）を取得
    rank = dist.get_rank() if dist.is_initialized() else 0

    # 分散環境のワールドサイズ（プロセス数）を取得
    world_size = dist.get_world_size() if dist.is_initialized() else 1

    # 結果を格納するテンソルを0で初期化
    correct = torch.zeros(len(data), dtype=torch.float32, device=device)

    # 2) データの評価

    # 各プロセスが担当するデータを順番に処理
    # world_size=8の場合
    # rank 0 -> idx = 0, 8, 16, ...
    # rank 1 -> idx = 1, 9, 17, ...
    # rank 7 -> idx = 7, 15, 23, ...
    for idx in range(rank, len(data), world_size):

        # 1つのデータを評価
        is_correct = evaluate_example(idx, model, tokenizer, data, device, task_meta)

        # 対応するインデックスに結果を格納
        correct[idx] = float(is_correct)

    # 3) 結果の集約

    if world_size > 1:

        # 全プロセスが完了するまで待機
        dist.barrier()

        # 全プロセスの結果を集約（合計を計算）
        dist.all_reduce(correct, op=dist.ReduceOp.SUM)

    # 正解率の平均を計算
    mean_correct = correct.mean().item()

    return mean_correct

In [ ]:
def evaluate_model(model, tokenizer, device, max_per_task=-1):
    """
    COREベンチマークを実行する
    HellaSwag、 ARC-Easyなどの全22タスクを順番に実行し、正解率を計算し、総合スコアに集約する

    Args:
        model: 評価するモデル
        tokenizer: モデルのトークナイザー
        device: モデルの実行デバイス
        max_per_task (int): 各タスクあたりの最大サンプル数（-1で無制限）
    Returns:
        dict: 各タスクの正解率、センタリングされた正解率、およびCOREメトリックを含む辞書
            {
                "results": {task_label: accuracy, ...},
                "centered_results": {task_label: centered_accuracy, ...},
                "core_metric": core_metric
            }
    """
    logger.debug(f"COREベンチマークを実行 {max_per_task=}")

    # 1) 設定とメタデータの読み込み

    base_dir = get_base_dir()
    eval_bundle_dir = os.path.join(base_dir, "eval_bundle")
    data_base_path = os.path.join(eval_bundle_dir, "eval_data")

    # 実行するべきタスクのリストのパス
    config_path = os.path.join(eval_bundle_dir, "core.yaml")

    # 各タスクのメタデータのパス
    eval_meta_data = os.path.join(eval_bundle_dir, "eval_meta_data.csv")

    # 設定ファイルを読み込む
    with open(config_path, 'r') as f:
        config = yaml.safe_load(f)

    # タスクのリストを取得
    tasks = config['icl_tasks']

    # メタデータを読み込む
    eval_metadata = pd.read_csv(eval_meta_data)

    # 2) タスクごとに評価を実行

    # 生の正解率を保存する辞書
    results = {}

    # 補正済みの正解率を保存する辞書
    centered_results = {}

    # 各タスクを順番に評価
    for task in tasks:

        start_time = time.time()
        label = task['label']
        task_meta = {
            'task_type': task['icl_task_type'],
            'dataset_uri': task['dataset_uri'],
            'num_fewshot': task['num_fewshot'][0],
            'continuation_delimiter': task.get('continuation_delimiter', ' ')
        }
        logger.info(f"Evaluating: {label} ({task_meta['num_fewshot']}-shot, type: {task_meta['task_type']})... ")

        # 現在のタスクのデータを読み込む
        data_path = os.path.join(data_base_path, task_meta['dataset_uri'])
        with open(data_path, 'r') as f:
            data = [json.loads(line.strip()) for line in f]

        # 一部のデータだけを使用するためシャッフル
        shuffle_rng = random.Random(1337)
        shuffle_rng.shuffle(data)

        # データを先頭から抽出
        if max_per_task > 0:
            data = data[:max_per_task]

        # タスクを評価し、正解率を取得
        accuracy = evaluate_task(model, tokenizer, data, device, task_meta)

        # タスクに対する正解率を保存
        results[label] = accuracy

        # 補正済みの正解率を計算
        row = eval_metadata[eval_metadata["Eval Task"] == label]
        random_baseline = row["Random baseline"].values[0]
        centered_result = (accuracy - 0.01 * random_baseline) / (1.0 - 0.01 * random_baseline)

        # タスクに対する補正済みの正解率を保存
        centered_results[label] = centered_result

        end_time = time.time()

        logger.info(f"accuracy: {accuracy:.4f} | centered: {centered_result:.4f} | time: {end_time - start_time:.2f}s")

    # 3) 結果を集計

    # センタリング済みの正解率を平均化
    core_metric = sum(centered_results.values()) / len(centered_results)

    # 結果を整形
    out = {
        "results": results,
        "centered_results": centered_results,
        "core_metric": core_metric
    }
    return out

### 訓練の実行

In [ ]:
# 訓練ループ
logger.setLevel(logging.INFO)

# BPBの評価結果を初期化
min_val_bpb = float("inf")

# 訓練損失の指数移動平均（EMA）を初期化
smooth_train_loss = 0

# EMAの減衰係数
ema_beta = 0.9

# 総訓練時間（wall-clock time）を初期化
total_training_time = 0

# 訓練ループの開始（+1は、最後に評価と保存を行うため）
for step in range(num_iterations + 1):

    # 1) 評価フェーズ

    # 最終ステップの判定
    last_step = step == num_iterations

    # これまでのFLOPsを計算
    flops_so_far = num_flops_per_token * total_batch_size * step

    # BPB評価を行う場合（すべてのランクが対象）
    if last_step or step % eval_every == 0:

        # モデルを評価モードに設定
        model.eval()

        # 検証データローダーを作成
        val_loader = build_val_loader()

        # 評価ステップ数を計算
        eval_steps = eval_tokens // (device_batch_size * max_seq_len * ddp_world_size)

        # 自動混合精度（Auto Mixed Precision, AMP）を有効化
        with autocast_ctx:
            # BPBを評価
            val_bpb = evaluate_bpb(model, val_loader, eval_steps, token_bytes)

        logger.info(f"ステップ数 {step=:05d} 検証BPB {val_bpb=:.4f}")

        # BPBが改善した場合
        if val_bpb < min_val_bpb:

            # 最小BPBを更新
            min_val_bpb = val_bpb

        # WandBにログを記録
        wandb_run.log({
            "step": step,
            "total_training_flops": flops_so_far,
            "total_training_time": total_training_time,
            "val/bpb": val_bpb,
        })

        # 訓練モードに戻す
        model.train()

    # COREメトリックの評価結果を初期化
    results = {}

    # COREメトリック評価を行う場合（すべてのランクが対象）
    if core_metric_every > 0 and (last_step or (step > 0 and step % core_metric_every == 0)):

        # モデルを評価モードに設定
        model.eval()

        with autocast_ctx:
            # COREメトリックを評価
            # 注意: 入力の形状が変化するため、コンパイル前のモデルを使用
            results = evaluate_model(orig_model, tokenizer, device, max_per_task=core_metric_max_per_task)

        logger.info(f"ステップ数 {step=:05d} | COREメトリック {results['core_metric']:.4f}")

        # WandBにログを記録
        wandb_run.log({
            "step": step,
            "total_training_flops": flops_so_far,
            "core_metric": results["core_metric"],
            "centered_results": results["centered_results"],
        })

        # 訓練モードに戻す
        model.train()

    # テキスト生成のサンプリングを行う場合（マスタープロセスのみ）
    if master_process and (last_step or (step > 0 and step % sample_every == 0)):

        # モデルを評価モードに設定
        model.eval()

        # サンプリング用のプロンプトを作成
        prompts = [
            "The capital of France is",
            "The chemical symbol of gold is",
            "If yesterday was Friday, then tomorrow will be",
            "The opposite of hot is",
            "The planets of the solar system are:",
            "My favorite color is",
            "If 5*x + 3 = 13, then x is",
        ]

        # 推論エンジンを初期化
        # 注意: 入力の形状が変化するため、コンパイル前のモデルを使用
        engine = Engine(orig_model, tokenizer)

        # 各プロンプトに対してテキストを生成
        for prompt in prompts:

            # トークン化
            tokens = tokenizer(prompt, prepend="<|bos|>")

            # 自動混合精度を有効化
            with autocast_ctx:

                # テキストを生成
                sample, _ = engine.generate_batch(
                    tokens,
                    num_samples=1,
                    max_tokens=16, # 16トークン生成
                    temperature=0 # 最も確率の高いトークンを選択
                )

            decoded = tokenizer.decode(sample[0])
            logger.info(f"ステップ数 {step=:05d} サンプル {decoded=}")
            
        # 訓練モードに戻す
        model.train()

    # 最終ステップでチェックポイントを保存（マスタープロセスのみ）
    if master_process and last_step:

        # チェックポイントのディレクトリ名を決定
        output_dirname = model_tag if model_tag else f"d{depth}"

        # チェックポイントのディレクトリパスを作成
        checkpoint_dir = os.path.join(base_dir, "base_checkpoints", output_dirname)

        # チェックポイントを保存
        save_checkpoint(
            checkpoint_dir,
            step,
            orig_model.state_dict(),
            [opt.state_dict() for opt in optimizers],
            {
                "step": step,
                "val_bpb": val_bpb, # BPB評価結果
                "model_config": model_config_kwargs,
                "user_config": user_config, # ユーザー設定
                "device_batch_size": device_batch_size,
                "max_seq_len": max_seq_len,
            }
        )

    # 最終ステップの場合
    if last_step:
        break

    # 2) 訓練フェーズ

    # GPUの操作を同期
    synchronize()

    # 計算時間の計測開始
    t0 = time.time()

    # 勾配を蓄積（gradient accumulation）
    for micro_step in range(grad_accum_steps):

        # 自動混合精度を有効化（Auto Mixed Precision, AMP）
        with autocast_ctx:

            # 順伝播を実行し、損失を計算
            loss = model(x, y)

        # ログ用に訓練損失の値を取得
        train_loss = loss.detach()

        # 損失を蓄積回数で割って正規化
        loss = loss / grad_accum_steps # each .backward() is a grad sum => normalize loss here

        # 逆伝播を実行
        loss.backward()

        # GPUが順伝播・逆伝播を実行している間に、次のバッチをプリフェッチ
        x, y = next(train_loader)

    # 勾配クリッピングが有効な場合（gradient clipping）
    grad_clip_enabled = grad_clip > 0.0
    if grad_clip_enabled:
        # grad_clipを超えていれば、勾配をクリップ
        grad_norm_tensor = torch.nn.utils.clip_grad_norm_(orig_model.parameters(), grad_clip)

        # 勾配ノルムをCPUに転送してfloatに変換
        grad_norm = grad_norm_tensor.item()

    # 現在のステップの学習率の倍率を取得
    lrm = get_lr_multiplier(step)

    # 全ての最適化関数の学習率を更新
    for opt in optimizers:
        for group in opt.param_groups:
            group["lr"] = group["initial_lr"] * lrm

    # 現在のステップのMuonのモーメンタムを取得
    muon_momentum = get_muon_momentum(step)

    # Muonのモーメンタムを更新
    for group in muon_optimizer.param_groups:
        group["momentum"] = muon_momentum

    # 勾配を使ってパラメータを更新
    for opt in optimizers:
        opt.step()

    # 勾配をリセット
    model.zero_grad(set_to_none=True)

    # 全てのGPUの操作を同期
    synchronize()

    # 計算時間の計測終了
    t1 = time.time()
    dt = t1 - t0

    # 3. ロギング

    # 訓練損失を指数移動平均（EMA）で更新
    smooth_train_loss = ema_beta * smooth_train_loss + (1 - ema_beta) * train_loss.item() 

    # EMA the training loss

    # debias the EMA
    debiased_smooth_loss = smooth_train_loss / (1 - ema_beta**(step + 1))

    pct_done = 100 * step / num_iterations

    # スループット（トークン/秒）を計算
    tok_per_sec = int(total_batch_size / dt)

    # MFU（Model FLOPs Utilization）を計算
    # GPUの理論上の最大FLOPsに対して、実際に達成したFLOPsの割合
    flops_per_sec = num_flops_per_token * total_batch_size / dt
    # bfloat16 H100 SXM and without 2:4 sparsity
    promised_flops_per_sec_h100 = 989e12 * ddp_world_size
    mfu = 100 * flops_per_sec / promised_flops_per_sec_h100 # in %

    # 10ステップ目以降の場合
    if step > 10:
        # 総訓練時間を更新
        # 最初の10ステップはコンパイルなどで遅いため
        total_training_time += dt

    print_grad_norm = f" grad norm: {grad_norm:.4f} |" if grad_clip_enabled else ""

    logger.info(f"step {step:05d}/{num_iterations:05d} ({pct_done:.2f}%) | loss: {debiased_smooth_loss:.6f} |{print_grad_norm} lrm: {lrm:.2f} | dt: {dt * 1000:.2f}ms | tok/sec: {tok_per_sec:,} | mfu: {mfu:.2f} | total time: {total_training_time/60:.2f}m")

    # 100ステップごと
    if step % 100 == 0:

        # WandBにログを記録
        log_data = {
            "step": step,
            "total_training_flops": flops_so_far,
            "total_training_time": total_training_time,
            "train/loss": debiased_smooth_loss,
            "train/lrm": lrm,
            "train/dt": dt,
            "train/tok_per_sec": tok_per_sec,
            "train/mfu": mfu,
        }

        if grad_clip_enabled:
            log_data["train/grad_norm"] = grad_norm

        wandb_run.log(log_data)